In [1]:
# Libraries

import sys
from bs4 import BeautifulSoup
import Cleaning as cln
from collections import defaultdict, Counter
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.multiprocessing import get
from bokeh.io import export_png
from bokeh.models import ColumnDataSource, TableColumn, DataTable
import datetime
import Exploration as exp
from gensim.models import TfidfModel, ldamodel, LdaMulticore, CoherenceModel
from gensim.corpora import Dictionary
import gmaps
from importlib import reload
from itertools import product
reload(cln)
reload(exp)
from langdetect import detect, DetectorFactory
from IPython.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import multiprocessing
import nltk
import numpy as np
import os
import pandas as pd
import pyLDAvis
import pickle
import random
import re
import scipy
import seaborn as sns
import sklearn as skl
from sklearn import manifold, metrics, model_selection, preprocessing, decomposition
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import make_classification
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, KFold
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import label_binarize, LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.svm import SVC
from spellchecker import SpellChecker
import statistics
import string
import time
import Transformer as mytransformer
reload(mytransformer)

pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_columns', 186)

%matplotlib widget
# %matplotlib inline

D:\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
class progress_bar():
    
    def __init__(self, barLength = None, title = None, progress_lim = None, time_lim=None):
        self.first_time = datetime.datetime.now()
        if barLength is not None:
            self.barLength = 40
        else :
            self.barLength = barLength
        
        if title is None:
            self.title = ''
        else:
            self.title = title
            
        if progress_lim is not None:
            self.progress_lim = progress_lim
            self.step_progress = 1/self.progress_lim
            
            self.time_lim = datetime.timedelta(hours=10)
            self.step_time = 1
            
            self.lim_kind = 'progress'
        elif time_lim is not None:
            self.progress_lim = 0
            self.step_progress = 1
            
            self.time_lim = datetime.timedelta(seconds = time_lim)
            self.step_time = 1/time_lim
            
            self.lim_kind = 'time'
        else:
            self.progress_lim = 0.01
            self.time_lim = datetime.timedelta(hours=10)
            self.time_lim = 1
            self.lim_kind = 'progress'
            
        self.old_progress = 0
        self.old_timer = datetime.timedelta(seconds = 0)
        self.progress = 0
        self.timer = datetime.timedelta(seconds = 0)
        
    def set_progress(self, progress):
        self.old_progress = -1
        self.update_progress(progress)
        
    def reset(self, title=None, progress=None):
        self.first_time = datetime.datetime.now()
        self.old_progress = -1
        self.old_timer = datetime.timedelta(seconds = 0)
        if title is not None:
            self.title = title
        if progress is None:
            self.update_progress(0)
        else:
            self.update_progress(progress)
            
    def update_progress(self, progress):
        self.timer = datetime.datetime.now() - self.first_time
#         status = datetime.datetime.strptime(str(time_spent), "%H:%M:%S")
        status = str(self.timer).format("%H:%M:%.0S")
        if isinstance(progress, int):
            progress = float(progress)
        if not isinstance(progress, float):
            progress = 0
            status = "error: progress var must be float\r\n"
        if progress < 0:
            progress = 0
            status = "Halt...\r\n"
        if progress >= 1 and self.old_progress < 1:
            self.old_progress = progress
            progress = 1
            status = "Done (" + status + ")\r\n"
        
        block = int(round(self.barLength*progress))
        if ((self.lim_kind == 'progress') & (progress >= self.old_progress + self.progress_lim)) |\
            ((self.lim_kind == 'time') & (self.timer >= self.old_timer + self.time_lim)):
            
            self.old_progress = (progress*self.step_progress - (progress*self.step_progress % 1))/self.step_progress
            self.old_timer = datetime.timedelta(
                seconds = 
                ( 
                    self.timer.total_seconds()*self.step_time
                    - (self.timer.total_seconds()*self.step_time % 1)
                )
                /self.step_time
            )
            
            text = "\rProgress {0}: [{1}] {2:.2f}% {3:s}".format( self.title, "#"*block + "-"*(self.barLength-block), progress*100, status)
            sys.stdout.write(text)
            sys.stdout.flush()
            
def save_df_as_image(df, path, width=None, height=None, width_policy = None, height_policy = None, b_index=False):
    df_copy = df.copy()
    if b_index:
        df_copy.insert(0, '', df_copy.index.values)
    
    if width is None:
        width = 280
    if height is None:
        height = 280
    if height_policy is None:
        height_policy = "auto"
    if width_policy is None:
        width_policy = "auto"
        
    source = ColumnDataSource(df_copy)
    df_columns = df_copy.columns.values

    columns_for_table = []
    
    for column in df_columns:
        columns_for_table.append(TableColumn(field=column, title=column))

    data_table = DataTable(source = source,
                           columns = columns_for_table,
                           width = width,
                           height = height,
                           height_policy = height_policy,
                           width_policy = width_policy,
                           index_position = None)
    export_png(data_table, filename = path)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 1) Mise en forme du jeu de donné

Requet SQL :
``` mysql
SELECT Id, PostTypeId, ParentId, CreationDate, Score, ViewCount, Body, OwnerUserId, Title, Tags, AnswerCount, CommentCount, FavoriteCount, ClosedDate
        FROM posts
        WHERE ViewCount > 50
        AND (CreationDate BETWEEN '2020-01-01' AND '2020-01-31') AND AnswerCount > 0 AND CommentCount > 5 AND len(Tags) > 0
        ORDER BY CreationDate
```

Equilibré le nombre de post sur chaque mois et sur chaque année. 

In [3]:
# USING CSV file for data
# Method from : http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/prepare_data_2017.html

FILE_2019 = "./Data/Posts_2019.csv"
data_2019 = pd.read_csv(FILE_2019, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_2019, ' : ', os.stat(FILE_2019).st_size / 2**20, 'Mo, ', data_2019.shape)

FILE_2020 = "./Data/Posts_2020.csv"
data_2020 = pd.read_csv(FILE_2020, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_2020, ' : ', os.stat(FILE_2020).st_size / 2**20, 'Mo', data_2020.shape)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


./Data/Posts_2019.csv  :  83.79751873016357 Mo,  (42393, 14)
./Data/Posts_2020.csv  :  37.11684703826904 Mo (17090, 14)


In [4]:
DataRaw = pd.concat([data_2019, data_2020], axis = 0, ignore_index=True)
DataRaw['CreationDate'] = DataRaw['CreationDate'].astype("datetime64")
DataRaw['ClosedDate'] = DataRaw['ClosedDate'].astype("datetime64")
DataRaw.describe(include = 'all', datetime_is_numeric=True)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
count,5.948300e+04,59483.0,0.0,59483,59483.000000,5.948300e+04,59483,5.848000e+04,59483,59483,59483.000000,59483.000000,35825.000000,1995
unique,NaN,NaN,NaN,NaN,NaN,NaN,59483,NaN,59483,41177,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,<p>I am trying to figure out how to ...,NaN,Hidden size vs input size in RNN,<flutter><dart>,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,1,392,NaN,NaN,NaN,NaN
mean,5.798659e+07,1.0,NaN,2019-09-14 07:00:44.380444928,9.164165,1.217933e+04,NaN,7.208163e+06,NaN,NaN,2.737606,2.017736,3.079749,2019-12-29 06:25:43.028070144
min,5.399238e+07,1.0,NaN,2019-01-01 00:48:55,-13.000000,5.001000e+03,NaN,9.100000e+01,NaN,NaN,1.000000,0.000000,0.000000,2019-01-01 10:07:16
25%,5.548793e+07,1.0,NaN,2019-04-03 05:52:03.500000,2.000000,6.081000e+03,NaN,3.497671e+06,NaN,NaN,1.000000,0.000000,1.000000,2019-04-30 06:37:09.500000
50%,5.742721e+07,1.0,NaN,2019-08-09 09:31:42,5.000000,7.899000e+03,NaN,7.726327e+06,NaN,NaN,2.000000,1.000000,2.000000,2019-11-08 14:20:51
75%,5.996404e+07,1.0,NaN,2020-01-29 09:27:59.500000,10.000000,1.211200e+04,NaN,1.091122e+07,NaN,NaN,3.000000,3.000000,3.000000,2020-07-27 01:43:07
max,6.552585e+07,1.0,NaN,2020-12-31 22:00:47,716.000000,1.579886e+06,NaN,1.571046e+07,NaN,NaN,51.000000,50.000000,203.000000,2021-12-03 04:33:51


In [5]:
# USING CSV file for data
# Method from : http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx/notebooks/prepare_data_2017.html

FILE_01_trim = "./Data/Trimester_01_2020.csv"
data_01_trim = pd.read_csv(FILE_01_trim, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_01_trim, ' : ', os.stat(FILE_01_trim).st_size / 2**20, 'Mo, ', data_01_trim.shape)

FILE_02_trim = "./Data/Trimester_02_2020.csv"
data_02_trim = pd.read_csv(FILE_02_trim, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_02_trim, ' : ', os.stat(FILE_02_trim).st_size / 2**20, 'Mo, ', data_02_trim.shape)

FILE_03_trim = "./Data/Trimester_03_2020.csv"
data_03_trim = pd.read_csv(FILE_03_trim, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_03_trim, ' : ', os.stat(FILE_03_trim).st_size / 2**20, 'Mo, ', data_03_trim.shape)

FILE_04_trim = "./Data/Trimester_04_2020.csv"
data_04_trim = pd.read_csv(FILE_04_trim, encoding="utf-8", low_memory=False, error_bad_lines=False)
print(FILE_04_trim, ' : ', os.stat(FILE_04_trim).st_size / 2**20, 'Mo, ', data_04_trim.shape)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


./Data/Trimester_01_2020.csv  :  65.8931360244751 Mo,  (28150, 14)
./Data/Trimester_02_2020.csv  :  64.53187656402588 Mo,  (27350, 14)
./Data/Trimester_03_2020.csv  :  60.011178970336914 Mo,  (25141, 14)
./Data/Trimester_04_2020.csv  :  49.752318382263184 Mo,  (20760, 14)


In [6]:
DataRaw = pd.concat([data_01_trim, data_02_trim, data_03_trim, data_04_trim], axis = 0, ignore_index=True)
DataRaw['CreationDate'] = DataRaw['CreationDate'].astype("datetime64")
DataRaw['ClosedDate'] = DataRaw['ClosedDate'].astype("datetime64")
DataRaw.describe(include = 'all', datetime_is_numeric=True)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
count,1.014010e+05,101401.0,0.0,101401,101401.000000,101401.000000,101401,9.880300e+04,101401,101401,101401.000000,101401.000000,17663.000000,6232
unique,NaN,NaN,NaN,NaN,NaN,NaN,101398,NaN,101399,65155,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,<p>So I was given the following Grad...,NaN,Regular Expression,<c++>,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,2,1340,NaN,NaN,NaN,NaN
mean,6.237847e+07,1.0,NaN,2020-06-19 02:47:10.031853824,0.829765,636.926095,NaN,8.883046e+06,NaN,NaN,1.541760,8.373132,1.220008,2020-07-04 14:31:35.701379840
min,5.954924e+07,1.0,NaN,2020-01-01 00:09:38,-24.000000,51.000000,NaN,9.100000e+01,NaN,NaN,1.000000,6.000000,0.000000,2020-01-01 01:36:16
25%,6.077459e+07,1.0,NaN,2020-03-20 12:53:55,0.000000,84.000000,NaN,4.906254e+06,NaN,NaN,1.000000,6.000000,1.000000,2020-03-19 20:56:12.500000
50%,6.236035e+07,1.0,NaN,2020-06-13 13:22:53,0.000000,169.000000,NaN,1.022240e+07,NaN,NaN,1.000000,7.000000,1.000000,2020-07-09 07:51:17
75%,6.384698e+07,1.0,NaN,2020-09-11 12:19:37,1.000000,450.000000,NaN,1.291629e+07,NaN,NaN,2.000000,9.000000,1.000000,2020-10-08 15:33:47.249999872
max,6.555971e+07,1.0,NaN,2020-12-30 23:53:24,609.000000,414968.000000,NaN,1.571046e+07,NaN,NaN,51.000000,74.000000,179.000000,2021-12-23 22:43:59


In [7]:
DataRaw.head(10)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,59549242,1,NaN,2020-01-01 00:09:38,10,14089,<p>I just started on my Flutter jour...,12634357.0,Integrating Paypal payments in Flutt...,<flutter><paypal><payment-gateway>,3,6,1.0,NaT
1,59549292,1,NaN,2020-01-01 00:27:42,0,266,<p>How can I change this so that I c...,11059320.0,Filter list of files using an array ...,<powershell>,2,11,NaN,NaT
2,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"<p><div class=""snippet"" data-lang=""j...",12633966.0,The text isn't showing up and the ba...,<html><css>,1,12,NaN,NaT
3,59549318,1,NaN,2020-01-01 00:34:43,0,375,<p>There is an issue with Bootstrap ...,NaN,Can someone help me fix my code issu...,<html><bootstrap-4><safari>,1,6,NaN,NaT
4,59549388,1,NaN,2020-01-01 00:58:27,0,80,<p>I'm wondering why the height of f...,12069171.0,Why the height of my first row is mo...,<html><css>,2,6,NaN,NaT
5,59549397,1,NaN,2020-01-01 01:01:55,-6,84,<p>so I have this code that with a f...,12433880.0,my program wont work properly and my...,<c++><function><segmentation-fault>,1,6,NaN,2020-01-01 01:36:16
6,59549477,1,NaN,2020-01-01 01:28:58,0,459,<p>I'm learning the C language with ...,12352162.0,Character Constants and Initializati...,<c><char><int><constants>,2,8,NaN,NaT
7,59549514,1,NaN,2020-01-01 01:40:05,0,1193,<p>I am getting following error whil...,2538627.0,CORS issue: Angular 5 on AWS S3 & Cl...,<asp.net-core><amazon-s3><cors><angu...,2,11,0.0,NaT
8,59549593,1,NaN,2020-01-01 02:03:35,2,427,<p>I am new to ajax and was followin...,2654750.0,Ajax with javascript not working on ...,<javascript><php><ajax><wampserver>,1,10,NaN,NaT
9,59549661,1,NaN,2020-01-01 02:28:04,1,76,<p>I can't figure out for the life o...,12572660.0,:not selector is not working as expe...,<javascript><jquery><html>,2,7,NaN,NaT


In [8]:
print(DataRaw.loc[7,'Title'])
print(DataRaw.loc[7,'Body'])

CORS issue: Angular 5 on AWS S3 & Cloudfront and .net core webapi on IIS10 EC 2 windows server 2019
<p>I am getting following error while accessing Angular app through browser (chrome/Edge/Firefox/Opera). <em><strong>API request works on direct calls from well through Postman/Fiddler.</strong></em></p>
<p>So i think there something between Cloudfront and EC2 calls and CORS policy</p>
<p><a href="https://i.stack.imgur.com/hsxeF.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/hsxeF.png" alt="enter image description here" /></a></p>
<p><a href="https://i.stack.imgur.com/TD3qQ.png" rel="nofollow noreferrer"><img src="https://i.stack.imgur.com/TD3qQ.png" alt="enter image description here" /></a></p>
<p>Following is my set up.</p>
<p><em><strong>Angular Setup:</strong></em></p>
<ul>
<li>S3 Bucket with Public access Policy</li>
<li>Cloudfront with Behavior whitelisting Headers</li>
</ul>
<p><a href="https://i.stack.imgur.com/JTbgd.png" rel="nofollow noreferrer"><img src="ht

In [ ]:
qual = ['Body', 'Title', 'Tags']
df = DataRaw[qual]
df.loc[:,'Tags'] = df.loc[:,'Tags'].apply(lambda x: x.replace('<', '')).apply(lambda x: x[:-1].split('>'))
df.describe(include = 'all', datetime_is_numeric=True)

In [132]:
df = DataRaw.copy()
soup = BeautifulSoup(df.loc[48325,'Body'], 'html.parser')
text = soup.get_text()
text

"I am writing APIs in NestJS which have a set of common headers. I decided to use interceptors in order to append headers to outgoing requests. The headers do not get appended to the request and hence the request keeps on failing.\nInterceptor\nimport * as utils from '../utils/utils';\nimport {\n  CallHandler,\n  ExecutionContext,\n  Injectable,\n  NestInterceptor\n} from '@nestjs/common';\nimport { HEADERS } from '../middlewares/headers.constant';\nimport { Observable } from 'rxjs';\nimport { Request } from 'express';\nimport { DATA_PARTITION_ID } from '../app.constants';\n\n@Injectable()\nexport class HeadersInterceptor implements NestInterceptor {\n  intercept(context: ExecutionContext, next: CallHandler): Observable<void> {\n    const ctx = context.switchToHttp();\n    const request: Request = ctx.getRequest();\n\n    this.setHeaders(request);\n\n    return next.handle();\n  }\n\n  private setHeaders(request): void {\n    this.updateHeaders(request, HEADERS.ACCEPT, 'application/jso

In [133]:
selection = ['NN', 'NNS', 'NNP', 'NNPS']
punct = string.punctuation
punct_space = " "*len(punct)
digits_space = " "*len(string.digits)
stop_words = set(nltk.corpus.stopwords.words('english'))

text = text.translate(str.maketrans(punct, punct_space, ''))
text = text.translate(str.maketrans(string.digits, digits_space, ''))
text = text.encode('ascii',errors='ignore').decode('ascii')

while '  ' in text:
        text = text.replace('  ', ' ')

text
# tokens = nltk.word_tokenize(text)
# tagged = nltk.pos_tag(tokens)
# tagged

# vec = [token.lower() for (token, tag) in tagged if (tag in selection) & (len(token) > 1)& (token not in stop_words)]
# vec

'I am writing APIs in NestJS which have a set of common headers I decided to use interceptors in order to append headers to outgoing requests The headers do not get appended to the request and hence the request keeps on failing \nInterceptor\nimport as utils from utils utils \nimport \n CallHandler \n ExecutionContext \n Injectable \n NestInterceptor\n from nestjs common \nimport HEADERS from middlewares headers constant \nimport Observable from rxjs \nimport Request from express \nimport DATA PARTITION ID from app constants \n\n Injectable \nexport class HeadersInterceptor implements NestInterceptor \n intercept context ExecutionContext next CallHandler Observable void \n const ctx context switchToHttp \n const request Request ctx getRequest \n\n this setHeaders request \n\n return next handle \n \n\n private setHeaders request void \n this updateHeaders request HEADERS ACCEPT application json \n this updateHeaders request HEADERS CONTENT TYPE application json \n this updateHeaders re

In [81]:
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [5]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Blaise-
[nltk_data]     Maël\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Blaise-Maël\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [30]:
ps = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

tokens = nltk.word_tokenize(text)
tokens = [ps.stem(tok) for tok in tokens]

tagged = nltk.pos_tag(tokens)
tagged

[('i', 'NNS'),
 ('have', 'VBP'),
 ('thi', 'VBN'),
 ('method', 'NN'),
 ('but', 'CC'),
 ('i', 'NN'),
 ('am', 'VBP'),
 ('not', 'RB'),
 ('abl', 'JJ'),
 ('to', 'TO'),
 ('figur', 'VB'),
 ('out', 'RP'),
 ('which', 'WDT'),
 ('paramet', 'NN'),
 ('should', 'MD'),
 ('i', 'VB'),
 ('pass', 'NN'),
 ('in', 'IN'),
 ('postman', 'NN'),
 ('to', 'TO'),
 ('execut', 'VB'),
 ('thi', 'JJ'),
 ('method', 'NN'),
 ('.', '.'),
 ('i', 'VB'),
 ('sent', 'VBD'),
 ('the', 'DT'),
 ('bodi', 'NN'),
 ('in', 'IN'),
 ('form-data', 'NN'),
 ('of', 'IN'),
 ('postman', 'NN'),
 ('and', 'CC'),
 ('content-typ', 'NN'),
 ('is', 'VBZ'),
 ('multipart/form-data', 'JJ'),
 ('.', '.'),
 ('later', 'RB'),
 ('i', 'RB'),
 ('have', 'VBP'),
 ('to', 'TO'),
 ('send', 'VB'),
 ('the', 'DT'),
 ('file', 'NN'),
 ('as', 'RB'),
 ('well', 'RB'),
 ('that', 'IN'),
 ("'s", 'VBZ'),
 ('whi', 'NN'),
 ('i', 'NN'),
 ('am', 'VBP'),
 ('use', 'RB'),
 ('form-data', 'JJ'),
 ('.', '.'),
 ('searchstr', 'NN'),
 ('is', 'VBZ'),
 ('come', 'VBN'),
 ('out', 'RP'),
 ('be', 'VB

In [31]:
selection = ['NN', 'NNS', 'NNP', 'NNPS']
punct_space = " "*len(string.punctuation)
digits_space = " "*len(string.digits)

ps = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

text = text.translate(str.maketrans(string.punctuation, punct_space, ''))
text = text.translate(str.maketrans(string.digits, digits_space, ''))
while '  ' in text:
        text = text.replace('  ', ' ')
        
tokens = nltk.word_tokenize(text)
tagged = nltk.pos_tag(tokens)
tagged

vec = [token.lower() for (token, tag) in tagged if (tag in selection) & (len(token) > 1)]
vec = [ps.stem(tok) for tok in vec]
vec

['method',
 'paramet',
 'postman',
 'method',
 'bodi',
 'data',
 'postman',
 'content',
 'type',
 'form',
 'data',
 'file',
 'form',
 'data',
 'searchstr',
 'code',
 'valu',
 'calendar',
 'method',
 'requestmethod',
 'post',
 'farecalendarrespons',
 'getcalendar',
 'requestparam',
 'valu',
 'searchstr',
 'string',
 'httpservletrequest',
 'httprequest',
 'except',
 'calendarrequest',
 'calenderrequest',
 'return',
 'farecalendarhandl',
 'builder',
 'httprequest',
 'httprequest',
 'request',
 'build',
 'getrespons']

In [103]:
selection = ['NN', 'NNS', 'NNP', 'NNPS']

vec = [token.lower() for (token, tag) in tagged if tag in selection]
vec

['method',
 'parameters',
 'postman',
 'method',
 'body',
 'data',
 'postman',
 'content',
 'type',
 'form',
 'data',
 'file',
 'form',
 'data',
 'searchstring',
 'code',
 'value',
 'calendar',
 'method',
 'requestmethod',
 'post',
 'farecalendarresponse',
 'getcalendar',
 'requestparam',
 'value',
 'searchstring',
 'string',
 'httpservletrequest',
 'httprequest',
 'exception',
 'calendarrequest',
 'calenderrequest',
 'return',
 'farecalendarhandler',
 'builder',
 'httprequest',
 'httprequest',
 'request',
 'build',
 'getresponse']

In [65]:
spell = SpellChecker()
[spell.correction(word) for word in vec]

['docker',
 'containers',
 'mac',
 'host',
 'container',
 'docker',
 'hub',
 'container',
 'docker',
 'hub',
 'containers',
 'access',
 'jenkins',
 'host',
 '//localhost:8080/',
 'access',
 'sonarqube',
 '//localhost:9000/',
 'jenkins',
 'container',
 'docker',
 'ad',
 'up',
 'jenkins/jenkins',
 'sonarqube',
 'container',
 'docker',
 'ad',
 'up',
 'sonarqube',
 'container',
 'communicate',
 'ip',
 'address',
 'container',
 'container',
 'ip',
 'address',
 'container',
 'docker',
 'inspect',
 'format',
 'container_name_or_id',
 'ip',
 'address',
 'jenkins',
 'container',
 'sonarqube',
 'container',
 'access',
 'jenkins',
 'container',
 'host',
 '//172.17.0.2:8080',
 'request',
 'timeout',
 'thing',
 'access',
 'sonarqube',
 'container',
 'host',
 '//172.17.0.3:9000',
 'behavior',
 'access',
 'container',
 'host',
 'ip',
 'address',
 'container',
 'jenkins',
 'access',
 'container',
 'sonarqube',
 'ip',
 'address']

In [8]:
# df.loc[0,'Title']
soup = BeautifulSoup(df.loc[65,'Title'], 'html.parser')
title = soup.get_text()
title

'Puppeteer Error: Chromium revision is not downloaded'

In [8]:
def set_str(row, bar=None, lenght = None, lemmatizer = None, stem = None):
    errors_counts = 0
    selection = ['NN', 'NNS', 'NNP', 'NNPS']
    stop_words = set(nltk.corpus.stopwords.words('english'))
    
    punct = string.punctuation
    punct_space = " "*len(string.punctuation)
    
    digits_space = " "*len(string.digits)

    if lenght is not None:
        progress = row.name / lenght
    else:
        progress = 1
    
    vec_title = []
    vec_body = []
        
    title = BeautifulSoup(row['Title'], 'html.parser').get_text()
    body = BeautifulSoup(row['Body'], 'html.parser').get_text()
    
    title = title.translate(str.maketrans(punct, punct_space,''))
    body = body.translate(str.maketrans(punct, punct_space,''))

    title = title.translate(str.maketrans(string.digits, digits_space,''))
    body = body.translate(str.maketrans(string.digits, digits_space,''))
    
    title = title.encode('ascii',errors='ignore').decode('ascii')
    body = body.encode('ascii',errors='ignore').decode('ascii')

    while '  ' in title:
        title = title.replace('  ', ' ')

    while '  ' in body:
        body = body.replace('  ', ' ')
                
    try:
        if detect(body) == 'en':

            # Suppression des mots qui ne sont pas des noms
            tok_title = nltk.word_tokenize(title)
            tok_body = nltk.word_tokenize(body)

            tag_title = nltk.pos_tag(tok_title)
            tag_body = nltk.pos_tag(tok_body)

            vec_title = [lemmatizer.lemmatize(token.lower()) for (token, tag) in tag_title if (tag in selection) & (len(token) > 1)& (token not in stop_words)]
            vec_body = [lemmatizer.lemmatize(token.lower()) for (token, tag) in tag_body if (tag in selection) & (len(token) > 1) & (token not in stop_words)]
        
    except:
        print("\nThis body and langag detect error: \n {0:s}\n {1:s}\n".format(row['Body'], body))
        errors_counts += 0
        
    row['Title'] = vec_title
    row['Body'] = vec_body

    if bar is not None:
        bar.update_progress(progress)
        
    return row

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
DetectorFactory.seed = 0
# nltk.download('stopwords')
# nltk.download('wordnet')

ps = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

ProgressBar().register()
ddata = dd.from_pandas(DataRaw, npartitions=4*multiprocessing.cpu_count())
with ProgressBar():
    DataModel = ddata.map_partitions(
        lambda df: df.apply(
            set_str,
            args = (None, len(df), lemmatizer), axis=1)
    ).compute(scheduler='processes')

DataModel = DataModel[DataModel.astype(str)['Title'] != '[]'] 

DataModel.describe(include='all', datetime_is_numeric=True)

[########################################] | 100% Completed | 16min 27.5s
[########################################] | 100% Completed | 16min 27.6s


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
count,1.008800e+05,100880.0,0.0,100880,100880.000000,100880.000000,100880,9.830300e+04,100880,100880,100880.000000,100880.000000,17551.000000,6167
unique,NaN,NaN,NaN,NaN,NaN,NaN,100877,NaN,96017,64898,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,NaN,"[gradedactivity, class, class, grade...",NaN,[function],<c++>,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,99,1325,NaN,NaN,NaN,NaN
mean,6.237787e+07,1.0,NaN,2020-06-19 01:54:48.374841088,0.828648,638.399375,NaN,8.876590e+06,NaN,NaN,1.541564,8.372224,1.220956,2020-07-04 17:08:41.918274560
min,5.954924e+07,1.0,NaN,2020-01-01 00:09:38,-24.000000,51.000000,NaN,9.100000e+01,NaN,NaN,1.000000,6.000000,0.000000,2020-01-01 01:36:16
25%,6.077349e+07,1.0,NaN,2020-03-20 11:45:01.249999872,0.000000,84.000000,NaN,4.889224e+06,NaN,NaN,1.000000,6.000000,1.000000,2020-03-19 18:23:26.500000
50%,6.235985e+07,1.0,NaN,2020-06-13 12:34:21.500000,0.000000,169.000000,NaN,1.021119e+07,NaN,NaN,1.000000,7.000000,1.000000,2020-07-09 13:18:47
75%,6.384517e+07,1.0,NaN,2020-09-11 10:19:59.249999872,1.000000,452.000000,NaN,1.291349e+07,NaN,NaN,2.000000,9.000000,1.000000,2020-10-08 19:45:45
max,6.555971e+07,1.0,NaN,2020-12-30 23:53:24,609.000000,414968.000000,NaN,1.571046e+07,NaN,NaN,51.000000,74.000000,179.000000,2021-12-23 22:43:59


In [ ]:
bar = progress_bar(10, 'treatement', 1e-2)
# bar = progress_bar(10, 'treatement', time_lim=1)
bar.set_progress(0)

lemmatizer = nltk.stem.WordNetLemmatizer()
DataModel = DataRaw.apply(set_str, args=(bar, len(DataRaw), lemmatizer), axis = 1)
DataModel = DataModel[DataModel.astype(str)['Title'] != '[]']
bar.update_progress(1)

DataModel.describe(include='all', datetime_is_numeric=True)

Progress treatement: [###-------------------------------------] 8.00% 0:01:58.707909
This body and langag detect error: 
 <p>I am making a single page in Angular 6. It a simple page and I would like to highlight <code>navbar</code> depending on scroll.</p>

<p>here is my code:</p>

<p><div class="snippet" data-lang="js" data-hide="false" data-console="true" data-babel="false">
<div class="snippet-code">
<pre class="snippet-code-css lang-css prettyprint-override"><code>.sticky {
  position: sticky;
  top: 0;
}
#i ul {
  list-style-type: none;
  margin: 0;
  padding: 0;
  overflow: hidden;
  background-color: blue;
}

#i li {
  float: left;
}

#i li a {
  display: block;
  color: white;
  text-align: center;
  padding: 14px 16px;
  text-decoration: none;
}

#i li a:hover {
  border-radius: 0px 0px 10px 10px;
  background-color: rgb(43, 137, 226);
}

/* #i {
  margin: 0px 0px 40px 0px;
} */

#footer-id {
  background-color:blue;
}</code></pre>
<pre class="snippet-code-html lang-html prett

Progress treatement: [####------------------------------------] 9.00% 0:02:13.485168
This body and langag detect error: 
 <p>I have a problem with LazyInitializationException and I don't know how to fix it.</p>

<pre><code>for (Long id : employeeIds)
    {
        List&lt;ProjectEmployee&gt; projectEmployeeList = projectEmployeeService.findProjectEmployeesWithinDates(id,
                startDate, endDate);

        // if no data, then continue with next employee
        if (projectEmployeeList.isEmpty())
        {
            continue;
        }

        gridCreated = true;

        Employee employee = projectEmployeeList.get(0).getEmployee();
        Label titleLabel = new Label(employee.getPerson().getSurname() + " " + employee.getPerson().getName() + " ["
                                     + employee.getRole().getHumanizedRole() + "]");
        titleLabel.setStyleName("header-bold");

        ProjectEmployeePanel projectEmployeePanel = new ProjectEmployeePanel(id, startDate, endD

Progress treatement: [####------------------------------------] 10.00% 0:02:29.195141

In [9]:
FILE_Model = "./Data/Model_v2.csv"
# DataModel.to_csv(FILE_Model, index=False, doublequote=False)

DataModel = pd.read_csv(FILE_Model, encoding="utf-8", low_memory=False, doublequote=False, error_bad_lines=False)
DataModel['Body'] = DataModel['Body'].astype('str')
DataModel['Body'] = DataModel.Body.apply(
    lambda x: 
    x.replace('\'', '')
    .replace('[', '')
    .replace(']', '')
).apply(
    lambda x: x[:].split(', ')
)
DataModel['Title'] = DataModel.Title.apply(
    lambda x: 
    x.replace('\'', '')
    .replace('[', '')
    .replace(']', '')
).apply(
    lambda x: x[:].split(', ')
)

print(DataModel['Body'].dtypes)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


object


In [7]:
multiprocessing.cpu_count()

8

In [10]:
DataModel['Tags'] = DataModel['Tags'].apply(lambda x: x.replace('<', '')).apply(lambda x: x[:-1].split('>'))

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
DataModel

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,59549242,1,NaN,2020-01-01 00:09:38,10,14089,"[flutter, journey, need, payment, fl...",12634357.0,"[payment, flutter, app]","[flutter, paypal, payment-gateway]",3,6,1.0,NaN
1,59549292,1,NaN,2020-01-01 00:27:42,0,266,"[cycle, list, file, array, expressio...",11059320.0,"[filter, list, file, array, expression]",[powershell],2,11,NaN,NaN
2,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"[body, background, color, size, px, ...",12633966.0,"[text, background, color, text, color]","[html, css]",1,12,NaN,NaN
3,59549318,1,NaN,2020-01-01 00:34:43,0,375,"[issue, bootstrap, safari, mac, col,...",NaN,"[someone, code, issue, bootstrap, sa...","[html, bootstrap-4, safari]",1,6,NaN,NaN
4,59549388,1,NaN,2020-01-01 00:58:27,0,80,"[height, row, march, product, jiko, ...",12069171.0,"[height, row, row]","[html, css]",2,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100884,65514630,1,NaN,2020-12-30 23:02:42,-2,96,"[problem, piece, chessboard, king, w...",14687335.0,"[program, queen, king]",[c],1,10,NaN,NaN
100885,65514632,1,NaN,2020-12-30 23:02:54,-2,57,"[middle, calendar, javascript, date,...",NaN,"[calendar, javascript]",[javascript],2,6,NaN,NaN
100886,65514712,1,NaN,2020-12-30 23:13:53,0,265,"[join, table, problem, table, table,...",15040818.0,"[jpa, jpql, statement]","[java, mysql, hibernate, jpa, jpql]",2,7,NaN,NaN
100887,65514890,1,NaN,2020-12-30 23:37:04,1,225,"[version, function, isalpha, work, l...",4668.0,"[glibc, isalpha, function, u, utf, l...","[c, locale, glibc, utf]",1,8,NaN,NaN


In [74]:
type(DataModel.loc[490, 'Body'])

list

In [10]:
feat = 'Body'

lookedfor = [['aaiaaaiaaaiaaaiaaaiaaaiaaaiaaaiaaaiaaaiaaaiaaamaaaiaaaiaaa']]
# lookedfor = ['abcdefghijklmonpqrstuvwxyz']
# lookedfor = [string.punctuation]
def check_words_list(column, lookedfor):
    return list(set(column) & set(lookedfor))
    
test = pd.DataFrame(DataModel[feat].apply(check_words_list, args=(lookedfor)))
test = test[test.astype(str)[feat] != '[]']
test

,Body


In [170]:
DataRaw.loc[490, 'Body']

'<p>I am trying to send a picture from my Pi camera to a flask web server. When I encode the image Base64 it doesn\'t seem to produce a valid image.</p>\n\n<p>I can take the picture and process it through <code>opencv</code>. The Base64 encoded image is passed to the web page, but the string sent is not a valid image. To prove this I have saved the image and processed it with an online Base64 converter. Pasting this string into the web page shows the image. </p>\n\n<pre><code>def Take_Picture(camera):\n    stream = io.BytesIO()                                           # saving the picture to an in-program stream \n    camera.resolution = (160,120)                                   # set resolution\n    camera.capture(stream, format=\'jpeg\', use_video_port=True)      # capture into stream\n    mydata = np.fromstring(stream.getvalue(), dtype=np.uint8)       # convert image into numpy array\n    img = cv2.imdecode(mydata, -1)                                  # to opencv image\n    data 

In [171]:
DataRaw.loc[48325, 'Body']

"<p>I am writing APIs in NestJS which have a set of common headers. I decided to use interceptors in order to append headers to outgoing requests. The headers do not get appended to the request and hence the request keeps on failing.</p>\n<p><strong>Interceptor</strong></p>\n<pre><code>import * as utils from '../utils/utils';\nimport {\n  CallHandler,\n  ExecutionContext,\n  Injectable,\n  NestInterceptor\n} from '@nestjs/common';\nimport { HEADERS } from '../middlewares/headers.constant';\nimport { Observable } from 'rxjs';\nimport { Request } from 'express';\nimport { DATA_PARTITION_ID } from '../app.constants';\n\n@Injectable()\nexport class HeadersInterceptor implements NestInterceptor {\n  intercept(context: ExecutionContext, next: CallHandler): Observable&lt;void&gt; {\n    const ctx = context.switchToHttp();\n    const request: Request = ctx.getRequest();\n\n    this.setHeaders(request);\n\n    return next.handle();\n  }\n\n  private setHeaders(request): void {\n    this.updateH

## 2) Analyse du dataset

### 2.A) Analyse univariée 

In [11]:
DataModel.plot(x='Id', y='AnswerCount', kind='scatter')

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='Id', ylabel='AnswerCount'>

In [12]:
DataModel.plot(x='Id', y='CommentCount', kind='scatter')

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='Id', ylabel='CommentCount'>

In [13]:
DataModel = DataModel[DataModel['CommentCount'] > 5]
DataModel = DataModel.reset_index()
DataModel

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,0,59549242,1,NaN,2020-01-01 00:09:38,10,14089,"[flutter, journey, need, payment, fl...",12634357.0,"[payment, flutter, app]","[flutter, paypal, payment-gateway]",3,6,1.0,NaN
1,1,59549292,1,NaN,2020-01-01 00:27:42,0,266,"[cycle, list, file, array, expressio...",11059320.0,"[filter, list, file, array, expression]",[powershell],2,11,NaN,NaN
2,2,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"[body, background, color, size, px, ...",12633966.0,"[text, background, color, text, color]","[html, css]",1,12,NaN,NaN
3,3,59549318,1,NaN,2020-01-01 00:34:43,0,375,"[issue, bootstrap, safari, mac, col,...",NaN,"[someone, code, issue, bootstrap, sa...","[html, bootstrap-4, safari]",1,6,NaN,NaN
4,4,59549388,1,NaN,2020-01-01 00:58:27,0,80,"[height, row, march, product, jiko, ...",12069171.0,"[height, row, row]","[html, css]",2,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100884,100884,65514630,1,NaN,2020-12-30 23:02:42,-2,96,"[problem, piece, chessboard, king, w...",14687335.0,"[program, queen, king]",[c],1,10,NaN,NaN
100885,100885,65514632,1,NaN,2020-12-30 23:02:54,-2,57,"[middle, calendar, javascript, date,...",NaN,"[calendar, javascript]",[javascript],2,6,NaN,NaN
100886,100886,65514712,1,NaN,2020-12-30 23:13:53,0,265,"[join, table, problem, table, table,...",15040818.0,"[jpa, jpql, statement]","[java, mysql, hibernate, jpa, jpql]",2,7,NaN,NaN
100887,100887,65514890,1,NaN,2020-12-30 23:37:04,1,225,"[version, function, isalpha, work, l...",4668.0,"[glibc, isalpha, function, u, utf, l...","[c, locale, glibc, utf]",1,8,NaN,NaN


In [147]:
# DataRaw.loc[0,'Body']

In [22]:
top = 50
# count, words = exp.most_common_words(DataModel['Body'], ', ', top)
# pd.DataFrame([words, count])
df = exp.check_common_words(DataModel['Body'], ', ', top)


D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

On regarde ici les 50 mots les plus fréquent dans le corps de text et aucun n'est étrange.

In [26]:
df = DataModel.copy()
feat = 'Body'
bins = 500
binwidth = 1
ex_thr = 0.1

Lenght = []
integral = 0
bin_min = 0
int_min = 0 
bin_max = 0
int_max = 0
min_done = False
max_done = False

fig = plt.figure(figsize=(10,5))
plt.title('Lenght of {0:s}'.format(feat))
plt.xlabel('Number of words')
plt.ylabel('Number of documents')

Lenght = df[feat].apply(lambda x : len(x) )
auc = len(Lenght)
mean = statistics.mean(Lenght)
stdev = statistics.stdev(Lenght)
print('Mean of distribution = {0:.2f}({1:.2f})'.format(mean,stdev))
print('Total integral : {0:.2f}'.format(auc))

# y, x, _ = plt.hist(Lenght, bins=range(min(Lenght), max(Lenght) + binwidth, binwidth))
y, x, _ = plt.hist(Lenght, bins=bins, range=(0, bins))

for ibin, content in enumerate(y):
    integral += content
#     print(integral, content, auc, (1 - ex_thr)*auc)
    if min_done == True and max_done == True:
        break
    if integral > (1 - ex_thr)*auc and max_done == False:
        bin_max = x[ibin]
        int_max = integral
        max_done = True
    if integral > ex_thr*auc and min_done == False:
        bin_min = x[ibin]
        int_min = integral
        min_done = True
    
print('{0:.0f}% integral : {1:.2f} at {2:.0f}'.format((1-ex_thr)*100, int_max, bin_max))
print('{0:.0f}% integral : {1:.2f} at {2:.0f}'.format(ex_thr*100, int_min, bin_min))
print('Max of distribution = {0:.2f}({1:.2f})'.format(max(y),np.argmax(y)))
    
plt.axvline(mean, color='red', linestyle='dashed', linewidth=2)
plt.text(mean, Lenght[int(mean)], 'Mean')

plt.axvline(np.argmax(y), color='black', linestyle='dashed', linewidth=2)
plt.text(np.argmax(y), max(y), 'Max')

plt.axvline(mean+stdev, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(mean-stdev, color='blue', linestyle='dashed', linewidth=1)
plt.text(mean+stdev, x[int(mean+stdev)], 'Mean +/- stdev')

plt.axvline(bin_max, color='green', linestyle='dashed', linewidth=1)
plt.text(bin_max, 0.8*max(y), '{0:.1f}-threshold'.format((1-ex_thr)*100))
plt.axvline(bin_min, color='green', linestyle='dashed', linewidth=1)
plt.text(bin_min, 0.8*max(y), '{0:.1f}-threshold'.format(ex_thr*100))
plt.show()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean of distribution = 117.09(164.89)
Total integral : 100889.00
90% integral : 90837.00 at 243
10% integral : 10871.00 at 22
Max of distribution = 1002.00(37.00)


Ici la moyenne est décentré par rapport à mod de la gaussienne. On voie que si on prend la moyenne plus ou moins 1 stdev on dépasse les 90% de la statistique. On préfèrera ici prendre un seuil qui correspond à 90% de la statistique soit les corps de text ayant moins de 243 mots

In [27]:
def select_row(row, feat, minimum, maximum):
    if minimum >= len(row[feat]) or len(row[feat]) >= maximum:
        row[feat] = np.nan
    return row

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
df = DataModel.copy()
feat = 'Body'
maximum = 243
minimum = 22

# ddata = dd.from_pandas(df, npartitions=4*multiprocessing.cpu_count())
# with ProgressBar():
#     df = ddata.map_partitions(
#         lambda df : df.apply(
#             select_row,
#             args=(feat, minimum, maximum),
#             axis=1
#         )
#     ).compute(scheduler='processes')

df = df.apply(select_row,
            args=(feat, minimum, maximum),
            axis=1)

DataReduced = df.dropna(subset=[feat])
DataReduced.describe()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,index,Id,PostTypeId,ParentId,Score,ViewCount,OwnerUserId,AnswerCount,CommentCount,FavoriteCount
count,79896.000000,7.989600e+04,79896.0,0.0,79896.000000,79896.000000,7.790200e+04,79896.000000,79896.000000,13788.000000
mean,50574.285784,6.238600e+07,1.0,NaN,0.806198,595.719360,8.801545e+06,1.533293,8.356639,1.195677
std,29129.211695,1.753194e+06,0.0,NaN,4.325216,3194.528955,4.582230e+06,0.944624,2.924564,2.157538
min,1.000000,5.954929e+07,1.0,NaN,-12.000000,51.000000,9.100000e+01,1.000000,6.000000,0.000000
25%,25397.750000,6.078524e+07,1.0,NaN,0.000000,84.000000,4.763651e+06,1.000000,6.000000,1.000000
50%,50638.500000,6.237202e+07,1.0,NaN,0.000000,166.000000,1.008557e+07,1.000000,7.000000,1.000000
75%,75784.250000,6.385348e+07,1.0,NaN,1.000000,437.000000,1.288766e+07,2.000000,9.000000,1.000000
max,100888.000000,6.555971e+07,1.0,NaN,609.000000,339670.000000,1.571046e+07,51.000000,74.000000,179.000000


In [29]:
top = 50
count, words = exp.most_common_words(DataModel['Title'], ', ', top)
pd.DataFrame([words, count])

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,file,value,function,error,python,array,data,list,object,string,class,code,java,number,type,javascript,column,way,element,sql,method,loop,time,json,image,table,react,php,j,api,query,server,line,app,variable,row,name,script,text,access,date,input,problem,laravel,property,user,return,page,form,html
1,6256,6190,6039,6014,5305,5112,4484,3717,3147,3046,3006,2717,2703,2562,2471,2359,2298,2155,2111,2085,2082,2059,2049,1959,1935,1812,1760,1737,1711,1627,1569,1554,1536,1521,1520,1511,1488,1486,1452,1421,1420,1370,1354,1345,1327,1266,1261,1254,1239,1229


In [30]:
df = DataReduced.copy()
feat = 'Title'
bins = 500
binwidth = 1
ex_thr = 0.1

Lenght = []
integral = 0
bin_min = 0
int_min = 0 
bin_max = 0
int_max = 0
min_done = False
max_done = False

fig = plt.figure(figsize=(10,5))
plt.title('Lenght of {0:s}'.format(feat))
plt.xlabel('Number of words')
plt.ylabel('Number of documents')

Lenght = df[feat].apply(lambda x : len(x) )
auc = len(Lenght)
mean = statistics.mean(Lenght)
stdev = statistics.stdev(Lenght)
print('Mean of distribution = {0:.2f}({1:.2f})'.format(mean,stdev))
print('Total integral : {0:.2f}'.format(auc))

y, x, _ = plt.hist(Lenght, bins=range(min(Lenght), max(Lenght) + binwidth, binwidth))
# y, x, _ = plt.hist(Lenght, bins=bins, range=(0, bins))

for ibin, content in enumerate(y):
    integral += content
#     print(integral, content, auc, (1 - ex_thr)*auc)
    if min_done == True and max_done == True:
        break
    if integral > (1 - ex_thr)*auc and max_done == False:
        bin_max = x[ibin]
        int_max = integral
        max_done = True
    if integral > ex_thr*auc and min_done == False:
        bin_min = x[ibin]
        int_min = integral
        min_done = True
    
print('{0:.0f}% integral : {1:.2f} at {2:.0f}'.format((1-ex_thr)*100, int_max, bin_max))
print('{0:.0f}% integral : {1:.2f} at {2:.0f}'.format(ex_thr*100, int_min, bin_min))
print('Max of distribution = {0:.2f}({1:.2f})'.format(max(y),np.argmax(y)))
    
plt.axvline(mean, color='red', linestyle='dashed', linewidth=2)
plt.text(mean, x[int(mean)], 'Mean')

plt.axvline(np.argmax(y), color='black', linestyle='dashed', linewidth=2)
plt.text(np.argmax(y), max(y), 'Max')

plt.axvline(mean+stdev, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(mean-stdev, color='blue', linestyle='dashed', linewidth=1)
plt.text(mean+stdev, x[int(mean+stdev)], 'Mean +/- stdev')

plt.axvline(bin_max, color='green', linestyle='dashed', linewidth=1)
plt.text(bin_max, 0.8*max(y), '{0:.1f}-threshold'.format((1-ex_thr)*100))
plt.axvline(bin_min, color='green', linestyle='dashed', linewidth=1)
plt.text(bin_min, 0.8*max(y), '{0:.1f}-threshold'.format(ex_thr*100))
plt.show()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean of distribution = 4.14(1.81)
Total integral : 79896.00
90% integral : 72153.00 at 6
10% integral : 13561.00 at 2
Max of distribution = 18701.00(3.00)


Ici les deux solutions converge vers 4 +/- 2.

In [283]:
top = 50
count, words = exp.most_common_words(DataModel['Tags'], ', ', top)
pd.DataFrame([words, count])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,python,javascript,c++,c#,java,c,php,html,reactjs,sql,android,python-3.x,arrays,node.js,css,mysql,angular,laravel,jquery,r,sql-server,json,swift,excel,typescript,pandas,vba,regex,ios,string,algorithm,flutter,django,spring-boot,.net,linux,bash,list,docker,asp.net-core,spring,firebase,vue.js,amazon-web-services,numpy,postgresql,pointers,multithreading,function,.net-core
1,14424,12270,9579,8710,7902,5566,4679,4266,3795,3171,3130,3109,3098,2629,2456,2419,2019,1955,1910,1891,1890,1872,1834,1677,1660,1636,1536,1463,1430,1309,1285,1267,1235,1221,1165,1140,1136,1056,1041,998,968,957,930,900,891,884,874,837,819,811


In [37]:
feat = 'Tags'
binwidth = 1
bins = 500
beacon_1 = 1000
beacon_2 = 500
beacon_3 = 100

x = []
y = []

count, words = exp.most_common_words(DataModel['Tags'], ', ', None)
auc = sum(count)

for i, value in enumerate(count):
    x.append(i)
    y.append(value/sum(count)*100)
    if value > beacon_1:
        x_1 = i
    if value > beacon_2:
        x_2 = i
    if value > beacon_3:
        x_3 = i

print('Number of tags : {0}'.format(len(words)))

fig = plt.figure(figsize=(10,5))
plt.title('{0:.0f} most frequent {1:s}'.format(bins,feat))
plt.xlabel('Tags rank')
plt.ylabel('Percent of documents')

# plt.hist(x, weights=y, bins=range(min(x), max(x) + binwidth, binwidth) )
plt.hist(x, weights=y, bins=bins, range=(0, bins))

plt.axvline(x_1, color='green', linestyle='dashed', linewidth=1)
plt.text(x_1, beacon_1/count[0], '{0}, {1}'.format(beacon_1, x_1))

plt.axvline(x_2, color='yellow', linestyle='dashed', linewidth=1)
plt.text(x_2, beacon_2/count[0], '{0}, {1}'.format(beacon_2, x_2))

plt.axvline(x_3, color='red', linestyle='dashed', linewidth=1)
plt.text(x_3, beacon_3/count[0], '{0}, {1}'.format(beacon_3, x_3))

plt.show()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of tags : 16248


<ipython-input-37-74caa5f55fb9>:26: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10,5))
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

On se fixe le minimum de docments par tag à 100, ce qui siginifie que l'on doit prendre les 369 tags les plus fréquents.

On a 16248 Tags différents dans le corpus.

In [80]:
fig = plt.figure(figsize=(10,9))
plt.bar(words, count)
plt.xticks(rotation=90)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

On regarde ici les 50 tags les plus fréquents.

In [38]:
df = DataReduced.copy()
feat = 'Tags'
bins = 6
binwidth = 1
ex_thr = 0.1

Lenght = []
integral = 0
bin_min = 0
int_min = 0 
bin_max = 0
int_max = 0
min_done = False
max_done = False

fig = plt.figure(figsize=(10,9))
plt.title('Lenght of {0:s}'.format(feat))

Lenght = df[feat].apply(lambda x : len(x) )
auc = len(Lenght)
mean = statistics.mean(Lenght)
stdev = statistics.stdev(Lenght)
print('Mean of distribution = {0:.2f}({1:.2f})'.format(mean,stdev))
print('Total integral : {0:.2f}'.format(auc))

# y, x, _ = plt.hist(Lenght, bins=range(min(Lenght), max(Lenght) + binwidth, binwidth))
y, x, _ = plt.hist(Lenght, bins=bins, range=(0, bins))

for ibin, content in enumerate(y):
    integral += content
#     print(integral, content, auc, (1 - ex_thr)*auc)
    if min_done == True and max_done == True:
        break
    if integral > (1 - ex_thr)*auc and max_done == False:
        bin_max = x[ibin]
        int_max = integral
        max_done = True
    if integral > ex_thr*auc and min_done == False:
        bin_min = x[ibin]
        int_min = integral
        min_done = True
    
print('{0:.0f}% integral : {1:.2f} at {2:.0f}'.format((1-ex_thr)*100, int_max, bin_max))
print('{0:.0f}% integral : {1:.2f} at {2:.0f}'.format(ex_thr*100, int_min, bin_min))
print('Max of distribution = {0:.2f}({1:.2f})'.format(max(y),np.argmax(y)))
    
plt.axvline(mean, color='red', linestyle='dashed', linewidth=2)
plt.text(mean, x[int(mean)], 'Mean')

plt.axvline(np.argmax(y), color='black', linestyle='dashed', linewidth=2)
plt.text(np.argmax(y), max(y), 'Max')

plt.axvline(mean+stdev, color='blue', linestyle='dashed', linewidth=1)
plt.axvline(mean-stdev, color='blue', linestyle='dashed', linewidth=1)
plt.text(mean+stdev, Lenght[int(mean+stdev)], 'Mean +/- stdev')

plt.axvline(bin_max, color='green', linestyle='dashed', linewidth=1)
plt.text(bin_max, 0.8*max(y), '{0:.1f}-threshold'.format((1-ex_thr)*100))
plt.axvline(bin_min, color='green', linestyle='dashed', linewidth=1)
plt.text(bin_min, 0.8*max(y), '{0:.1f}-threshold'.format(ex_thr*100))
plt.show()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean of distribution = 3.12(1.24)
Total integral : 79896.00
90% integral : 79896.00 at 5
10% integral : 8053.00 at 1
Max of distribution = 22294.00(3.00)


Pour l'instant on décide de ne pas réduire le data set sur la taille des tags.

In [40]:
top = 3
lst = [['premier','deuxieme','troisieme' ], ['premier','deuxieme','troisieme' ], ['premier','deuxieme','troisieme' ]]
df_test = pd.DataFrame({'Col1' : lst})
df_test
count, words = exp.most_common_words(df_test['Col1'], ', ', top)
pd.DataFrame([words, count])

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0,1,2
0,premier,deuxieme,troisieme
1,3,3,3


Vérification du bon fonctionnement de la fonction précédente.

In [41]:
top = 3
fig = plt.figure(figsize=(10,9))

plt.hist(x=words, weights=count, bins = top)
plt.title('The top {:d} of most frequent words'.format(top))
plt.xticks(rotation=90)
plt.show()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [42]:
def frequence_words(labels, delimiter):
    words = []
    unique = []
    count = []
    words_list = []
    for lab in labels:
        words_list = pd.Series(lab).unique().tolist()
        unique += exp.get_words(words_list, delimiter)
        
        words += exp.get_words(lab, delimiter)
    counter_w = Counter(words)
    counter_u = Counter(unique)
    words = []
    unique = []
    i = 0
    for word in counter_w:
        words.append(word)
        count.append(counter_w[word])
        unique.append(counter_u[word])
    
    df = pd.DataFrame({'Words' : words, 'Count' : count, 'Documents' : unique}).sort_values(by=['Count'], ascending=False)
    return df

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
top = 50
df_freq = frequence_words(DataReduced['Body'], ', ')
df_freq

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Words,Count,Documents
66,data,69272,21486
142,value,66049,22111
37,code,65574,36872
70,class,65423,20173
2,file,63943,18838
...,...,...,...
130512,anular,1,1
130511,sparkjob,1,1
130507,addcookies,1,1
130505,baseapicontroller,1,1


In [55]:
save_df_as_image(df_freq[:5], './Images/Words_freq.png')

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


On fait une fonction qui permet de voir les différents mots du corpus, leur nombre et leur présence dans les documents.

In [34]:
df_freq[df_freq['Documents'] < 2]

,Words,Count,Documents
66869,vrtx,102,1
194304,chembl,100,1
122337,datarace,71,1
175933,mahmood,63,1
80981,cutnumber,60,1
...,...,...,...
130512,anular,1,1
130511,sparkjob,1,1
130507,addcookies,1,1
130505,baseapicontroller,1,1


On regarde les documents présent dans un seul document, l'objectif sera de les enlever pour réduire les features, surtout que ces mots ne permettront pas de définir les sujets.

In [44]:
df_freq.plot(x='Count', y='Documents', kind='scatter')
# plt.yscale('log')
plt.show()

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Canvas).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
D:\Anaconda3\lib\site-packages\ipywidgets\widgets\widget.py:412: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Wi

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [45]:
def repeated_letters(df, feat, repeat, select='keep'):
    matcher = re.compile(r'(.)\1*')
    col = df[feat].values

    new_col = []
    for c in col:
        if select == 'unkeep':
            keep = True
        else :
            keep = False
    #     print(c)
        for match in matcher.finditer(c):
            if len(match.group())>=repeat:
                if select == 'unkeep':
                    keep = False
                else :
                    keep = True
                break
        if keep :
            new_col.append(c)
    return new_col

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
repeated = repeated_letters(df_freq, 'Words', 3)
len(repeated)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2059

In [38]:
repeated

['www',
 'xxx',
 'yyyy',
 'xxxx',
 'fff',
 'aaa',
 'bbb',
 'xxxxx',
 'ccc',
 'ffffff',
 'xxxxxx',
 'yyy',
 'ddd',
 'ffff',
 'aaaa',
 'xxxxxxx',
 'eee',
 'mmm',
 'iii',
 'ieee',
 'zzz',
 'adddbcontext',
 'xffff',
 'wwwroot',
 'yyyymmdd',
 'xxxxxxxxxx',
 'processstartinfo',
 'xxxxxxxxx',
 'xxxxxxxx',
 'xxxxxxxxxxxx',
 'readalllines',
 'xffffffff',
 'cccc',
 'zzzz',
 'issuccessstatuscode',
 'bbbb',
 'dddd',
 'xxxxxxxxxxx',
 'adddays',
 'mmmm',
 'hhh',
 'purrr',
 'jjj',
 'addresssanitizer',
 'ttt',
 'adddata',
 'xbffff',
 'ggg',
 'hrrr',
 'ensuresuccessstatuscode',
 'fffffff',
 'aaaaa',
 'sss',
 'aaaaaa',
 'fffff',
 'ppp',
 'vvv',
 'ffffffff',
 'fffffffe',
 'eeee',
 'rrr',
 'eeeeee',
 'bbbbb',
 'fwww',
 'cccccc',
 'lll',
 'dddddd',
 'installlocation',
 'xxxxxxxxxxxxxx',
 'currrequest',
 'kkk',
 'qqq',
 'apppath',
 'getxxxxx',
 'xxxxxxxxxxxxxxxx',
 'fffe',
 'sssz',
 'xxxxxxxxxxxxxxx',
 'zzzzz',
 'oooo',
 'xxxxxxxxxxxxx',
 'fffd',
 'xffffff',
 'naaaad',
 'xxxxxxxxxxxxxxxxxxxxx',
 'aaaaaaa',


Ici on regarde les mots qui on plus de 2 * la même lettre qui se suivent car on voudrait les supprimer en considérant que ces mots n'existe pas. La majorité des mots sont inintéressant mais quelques'un sont importants.

In [47]:
def no_vowel(df, feat, select='keep'):
    col = df[feat].values
    if select == 'unkeep':
        keep = False
    else :
        keep = True
            
    new_col = []
    for c in col:
        s = re.findall(r'\b[^AEIOUY_0-9\W]+\b', c, flags=re.I)
#         print(c)
        if keep:
            for w in s:
                new_col.append(w)
        elif not keep :
            new_col.append(c)
            
    return new_col

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
novowel = no_vowel(df_freq, 'Words')
novowel

['http',
 'j',
 'src',
 'html',
 'php',
 'td',
 'db',
 't',
 'm',
 'cs',
 'px',
 'www',
 'ssl',
 'xml',
 'df',
 'sdk',
 'npm',
 'sql',
 'jdk',
 'cv',
 'gm',
 'tf',
 'btn',
 'std',
 'dll',
 'pm',
 'th',
 'tcp',
 'cmd',
 'cpp',
 'rb',
 'str',
 'csv',
 'tr',
 'cc',
 'v',
 'ssh',
 'ff',
 'xxx',
 'jdbc',
 'mb',
 'cjs',
 'tl',
 'cd',
 'c',
 'xmlns',
 'png',
 'gcc',
 'tmp',
 'mvc',
 'msvc',
 'txt',
 'br',
 'md',
 'pdf',
 'mm',
 'plt',
 'dd',
 'msg',
 'pc',
 'jwt',
 'fff',
 'fb',
 'jpg',
 'kb',
 'pd',
 'vc',
 'xx',
 'rc',
 'bb',
 'jsx',
 'bf',
 'pcl',
 'p',
 'ng',
 'cf',
 'vm',
 'np',
 'mp',
 'bd',
 'gb',
 'sh',
 'dc',
 'f',
 'xc',
 'sm',
 'mt',
 'npmjs',
 'xsd',
 'smtp',
 'ch',
 'fc',
 'sc',
 'pg',
 'cp',
 'cl',
 'xxxx',
 'fn',
 'svg',
 'sdl',
 'csrf',
 'dp',
 'bc',
 'cb',
 'rw',
 'rt',
 'dl',
 'ctx',
 'l',
 'fd',
 'qt',
 'x',
 'ptr',
 'kt',
 'jvm',
 'sp',
 'w',
 'fp',
 'gz',
 'rm',
 'gmt',
 'st',
 'r',
 'xxxxxx',
 'tk',
 'httpd',
 'vpc',
 'pkg',
 'gh',
 'gd',
 'gc',
 'msb',
 'xhr',
 'xxxxx',

On regardes les mots sans voyelles en considérant qu'il n'existe pas et ne sont pas important. Les mots sans voyelles sont important !

In [48]:
u_doc = df_freq[df_freq['Documents'] < 2].loc[:,'Words'].values.tolist()
len(u_doc)

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


180233

In [49]:
def del_words(row, feat, words, bar=None, lenght=None):
    if lenght is not None:
        progress = row.name / lenght
    else:
        progress = 1
#     print('Flag 1')
    new_list = []
    new_list = [w for w in row[feat] if w not in words]
    row[feat] = new_list
#     print(new_list)
    if bar is not None:
        bar.update_progress(progress)
    return row

D:\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
df = DataReduced.copy()
bar = progress_bar(10, 'del_words', 1e-2)
# bar = progress_bar(10, 'treatement', time_lim=1)
bar.set_progress(0)

del_arrow = repeated
DataReduced = df.apply(
    del_words,
    args = ('Body', del_arrow, bar, len(df)),
    axis = 1
)

bar.update_progress(1)
DataReduced.describe()

Progress del_words: [##################################################] 126.00% 0:04:38.001619

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
1,59549292,1,NaN,2020-01-01 00:27:42,0,266,"[cycle, list, file, array, expressio...",11059320.0,"[filter, list, file, array, expression]",[powershell],2,11,NaN,NaN
2,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"[body, background, color, size, px, ...",12633966.0,"[text, background, color, text, color]","[html, css]",1,12,NaN,NaN
4,59549388,1,NaN,2020-01-01 00:58:27,0,80,"[height, row, march, product, jiko, ...",12069171.0,"[height, row, row]","[html, css]",2,6,NaN,NaN
5,59549397,1,NaN,2020-01-01 01:01:55,-6,84,"[code, function, number, array, powe...",12433880.0,"[program, wont, work, function, segm...","[c++, function, segmentation-fault]",1,6,NaN,2020-01-01 01:36:16
6,59549477,1,NaN,2020-01-01 01:28:58,0,459,"[language, book, primer, plus, curre...",12352162.0,"[character, constant, initialization]","[c, char, int, constants]",2,8,NaN,NaN
7,59549514,1,NaN,2020-01-01 01:40:05,0,1193,"[error, app, browser, chrome, edge, ...",2538627.0,"[cors, issue, angular, aws, cloudfro...","[asp.net-core, amazon-s3, cors, angu...",2,11,0.0,NaN
8,59549593,1,NaN,2020-01-01 02:03:35,2,427,"[tutorial, food, search, app, user, ...",2654750.0,"[ajax, javascript, wamp, server]","[javascript, php, ajax, wampserver]",1,10,NaN,NaN
9,59549661,1,NaN,2020-01-01 02:28:04,1,76,"[figure, life, reason, screen, regar...",12572660.0,"[selector, jquery]","[javascript, jquery, html]",2,7,NaN,NaN
10,59549750,1,NaN,2020-01-01 02:59:38,0,173,"[answer, post, file, size, image, fi...",12348447.0,"[file, size, image, gallery]",[android],5,6,NaN,NaN
11,59549824,1,NaN,2020-01-01 03:23:31,-2,74,"[problem, time, linux, root, user, j...",4280836.0,"[data, randomaccessfile, linux]","[java, linux, java-io, randomaccessf...",1,8,NaN,2020-01-01 08:50:42


In [45]:
# bar = progress_bar(10, 'del_words', 1e-2)
bar = progress_bar(10, 'treatement', time_lim=1)
bar.set_progress(0)

del_arrow = u_doc
DataReduced = DataReduced.apply(
    del_words,
    args = ('Body', del_arrow, bar, len(DataReduced)),
    axis = 1
)

bar.update_progress(1)
DataReduced.head(10)

Progress treatement: [----------------------------------------] 0.63% 0:05:40.220253

KeyboardInterrupt: 

In [46]:
ProgressBar().register()
ddata = dd.from_pandas(DataReduced, npartitions=4*multiprocessing.cpu_count())

del_arrow = u_doc
with ProgressBar():
    DataReduced = ddata.map_partitions(
        lambda df: df.apply(
            del_words,
            args=('Body', del_arrow),
            axis = 1
        )
    ).compute(scheduler='processes')
DataReduced.head(5)

[########################################] | 100% Completed |  2hr 34min 14.7s
[########################################] | 100% Completed |  2hr 34min 14.8s


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
1,59549292,1,NaN,2020-01-01 00:27:42,0,266,"[cycle, list, file, array, expressio...",11059320.0,"[filter, list, file, array, expression]",[powershell],2,11,NaN,NaN
2,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"[body, background, color, size, px, ...",12633966.0,"[text, background, color, text, color]","[html, css]",1,12,NaN,NaN
4,59549388,1,NaN,2020-01-01 00:58:27,0,80,"[height, row, march, product, eco, r...",12069171.0,"[height, row, row]","[html, css]",2,6,NaN,NaN
5,59549397,1,NaN,2020-01-01 01:01:55,-6,84,"[code, function, number, array, powe...",12433880.0,"[program, wont, work, function, segm...","[c++, function, segmentation-fault]",1,6,NaN,2020-01-01 01:36:16
6,59549477,1,NaN,2020-01-01 01:28:58,0,459,"[language, book, primer, plus, curre...",12352162.0,"[character, constant, initialization]","[c, char, int, constants]",2,8,NaN,NaN
7,59549514,1,NaN,2020-01-01 01:40:05,0,1193,"[error, app, browser, chrome, edge, ...",2538627.0,"[cors, issue, angular, aws, cloudfro...","[asp.net-core, amazon-s3, cors, angu...",2,11,0.0,NaN
8,59549593,1,NaN,2020-01-01 02:03:35,2,427,"[tutorial, food, search, app, user, ...",2654750.0,"[ajax, javascript, wamp, server]","[javascript, php, ajax, wampserver]",1,10,NaN,NaN
9,59549661,1,NaN,2020-01-01 02:28:04,1,76,"[figure, life, reason, screen, regar...",12572660.0,"[selector, jquery]","[javascript, jquery, html]",2,7,NaN,NaN
10,59549750,1,NaN,2020-01-01 02:59:38,0,173,"[answer, post, file, size, image, fi...",12348447.0,"[file, size, image, gallery]",[android],5,6,NaN,NaN
11,59549824,1,NaN,2020-01-01 03:23:31,-2,74,"[problem, time, linux, root, user, j...",4280836.0,"[data, randomaccessfile, linux]","[java, linux, java-io, randomaccessf...",1,8,NaN,2020-01-01 08:50:42


In [48]:
DataReduced.describe()

,Id,PostTypeId,ParentId,Score,ViewCount,OwnerUserId,AnswerCount,CommentCount,FavoriteCount
count,7.989600e+04,79896.0,0.0,79896.000000,79896.000000,7.790200e+04,79896.000000,79896.000000,13788.000000
mean,6.238600e+07,1.0,NaN,0.806198,595.719360,8.801545e+06,1.533293,8.356639,1.195677
std,1.753194e+06,0.0,NaN,4.325216,3194.528955,4.582230e+06,0.944624,2.924564,2.157538
min,5.954929e+07,1.0,NaN,-12.000000,51.000000,9.100000e+01,1.000000,6.000000,0.000000
25%,6.078524e+07,1.0,NaN,0.000000,84.000000,4.763651e+06,1.000000,6.000000,1.000000
50%,6.237202e+07,1.0,NaN,0.000000,166.000000,1.008557e+07,1.000000,7.000000,1.000000
75%,6.385348e+07,1.0,NaN,1.000000,437.000000,1.288766e+07,2.000000,9.000000,1.000000
max,6.555971e+07,1.0,NaN,609.000000,339670.000000,1.571046e+07,51.000000,74.000000,179.000000


Ici on supprime les mots contenue dans une liste.
On se retrouve avec 79896 individues et 15 varriables

In [3]:
FILE_Model = "./Data/DataCleaned_v1.csv"
# DataReduced.to_csv(FILE_Model, index=False, doublequote=False)

DataReduced = pd.read_csv(FILE_Model, encoding="utf-8", low_memory=False, doublequote=False, error_bad_lines=False)
DataReduced['Body'] = DataReduced['Body'].astype('str')
DataReduced['Body'] = DataReduced.Body.apply(
    lambda x: 
    x.replace('\'', '')
    .replace('[', '')
    .replace(']', '')
).apply(
    lambda x: x[:].split(', ')
)
DataReduced['Title'] = DataReduced.Title.apply(
    lambda x: 
    x.replace('\'', '')
    .replace('[', '')
    .replace(']', '')
).apply(
    lambda x: x[:].split(', ')
)
DataReduced['Tags'] = DataReduced.Tags.apply(
    lambda x: 
    x.replace('\'', '')
    .replace('[', '')
    .replace(']', '')
).apply(
    lambda x: x[:].split(', ')
)

DataReduced

,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,59549292,1,NaN,2020-01-01 00:27:42,0,266,"[cycle, list, file, array, expressio...",11059320.0,"[filter, list, file, array, expression]",[powershell],2,11,NaN,NaN
1,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"[body, background, color, size, px, ...",12633966.0,"[text, background, color, text, color]","[html, css]",1,12,NaN,NaN
2,59549388,1,NaN,2020-01-01 00:58:27,0,80,"[height, row, march, product, eco, r...",12069171.0,"[height, row, row]","[html, css]",2,6,NaN,NaN
3,59549397,1,NaN,2020-01-01 01:01:55,-6,84,"[code, function, number, array, powe...",12433880.0,"[program, wont, work, function, segm...","[c++, function, segmentation-fault]",1,6,NaN,2020-01-01 01:36:16
4,59549477,1,NaN,2020-01-01 01:28:58,0,459,"[language, book, primer, plus, curre...",12352162.0,"[character, constant, initialization]","[c, char, int, constants]",2,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79891,65514630,1,NaN,2020-12-30 23:02:42,-2,96,"[problem, piece, chessboard, king, w...",14687335.0,"[program, queen, king]",[c],1,10,NaN,NaN
79892,65514632,1,NaN,2020-12-30 23:02:54,-2,57,"[middle, calendar, javascript, date,...",NaN,"[calendar, javascript]",[javascript],2,6,NaN,NaN
79893,65514712,1,NaN,2020-12-30 23:13:53,0,265,"[join, table, problem, table, table,...",15040818.0,"[jpa, jpql, statement]","[java, mysql, hibernate, jpa, jpql]",2,7,NaN,NaN
79894,65514890,1,NaN,2020-12-30 23:37:04,1,225,"[version, function, isalpha, work, l...",4668.0,"[glibc, isalpha, function, u, utf, l...","[c, locale, glibc, utf]",1,8,NaN,NaN


### 2.B) Analyse bivariée 

## 3) Vectorization & Visualisation

In [4]:
def select_tags(row, words):
    if any( item in words for item in row['Tags'] ):
        return row
    else :
        row['Tags'] = np.nan
    return row

In [5]:
df = DataReduced.copy()

top = 369
count, words = exp.most_common_words(DataReduced['Tags'], ', ', top)
pd.DataFrame([words, count])

# DataTagged = df.apply(select_tags, args=([words]), axis = 1)

ProgressBar().register()
ddata = dd.from_pandas(df, npartitions=4*multiprocessing.cpu_count())

with ProgressBar():
    DataTagged = ddata.map_partitions(
        lambda df: df.apply(
            select_tags,
            args=([words]),
            axis = 1
        )
    ).compute(scheduler='processes')

DataTagged = DataTagged.dropna(subset=['Tags'])
# DataTagged.describe(include = 'all')
DataTagged

[########################################] | 100% Completed |  8.3s
[########################################] | 100% Completed |  8.4s


,Id,PostTypeId,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,59549292,1,NaN,2020-01-01 00:27:42,0,266,"[cycle, list, file, array, expressio...",11059320.0,"[filter, list, file, array, expression]",[powershell],2,11,NaN,NaN
1,59549299,1,NaN,2020-01-01 00:29:41,-2,477,"[body, background, color, size, px, ...",12633966.0,"[text, background, color, text, color]","[html, css]",1,12,NaN,NaN
2,59549388,1,NaN,2020-01-01 00:58:27,0,80,"[height, row, march, product, eco, r...",12069171.0,"[height, row, row]","[html, css]",2,6,NaN,NaN
3,59549397,1,NaN,2020-01-01 01:01:55,-6,84,"[code, function, number, array, powe...",12433880.0,"[program, wont, work, function, segm...","[c++, function, segmentation-fault]",1,6,NaN,2020-01-01 01:36:16
4,59549477,1,NaN,2020-01-01 01:28:58,0,459,"[language, book, primer, plus, curre...",12352162.0,"[character, constant, initialization]","[c, char, int, constants]",2,8,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79891,65514630,1,NaN,2020-12-30 23:02:42,-2,96,"[problem, piece, chessboard, king, w...",14687335.0,"[program, queen, king]",[c],1,10,NaN,NaN
79892,65514632,1,NaN,2020-12-30 23:02:54,-2,57,"[middle, calendar, javascript, date,...",NaN,"[calendar, javascript]",[javascript],2,6,NaN,NaN
79893,65514712,1,NaN,2020-12-30 23:13:53,0,265,"[join, table, problem, table, table,...",15040818.0,"[jpa, jpql, statement]","[java, mysql, hibernate, jpa, jpql]",2,7,NaN,NaN
79894,65514890,1,NaN,2020-12-30 23:37:04,1,225,"[version, function, isalpha, work, l...",4668.0,"[glibc, isalpha, function, u, utf, l...","[c, locale, glibc, utf]",1,8,NaN,NaN


In [15]:
words

['python',
 'javascript',
 'c++',
 'c#',
 'java',
 'c',
 'php',
 'html',
 'reactjs',
 'sql',
 'arrays',
 'python-3.x',
 'node.js',
 'android',
 'css',
 'mysql',
 'angular',
 'laravel',
 'json',
 'r',
 'sql-server',
 'jquery',
 'swift',
 'pandas',
 'typescript',
 'excel',
 'vba',
 'ios',
 'regex',
 'algorithm',
 'string',
 'bash',
 'flutter',
 '.net',
 'django',
 'linux',
 'asp.net-core',
 'docker',
 'list',
 'spring-boot',
 'vue.js',
 'amazon-web-services',
 'firebase',
 'numpy',
 'pointers',
 'multithreading',
 'postgresql',
 '.net-core',
 'function',
 'spring',
 'dataframe',
 'windows',
 'react-native',
 'mongodb',
 'azure',
 'loops',
 'ajax',
 'performance',
 'google-apps-script',
 'kotlin',
 'oracle',
 'powershell',
 'asp.net',
 'visual-studio',
 'git',
 'dart',
 'go',
 'csv',
 'for-loop',
 'dictionary',
 'linq',
 'class',
 'c++11',
 'shell',
 'kubernetes',
 'database',
 'express',
 'async-await',
 'sorting',
 'tkinter',
 'assembly',
 'file',
 'api',
 'google-sheets',
 'selenium',


In [6]:
df = DataTagged.copy()
corpora = df['Title'] + df['Body']

# Create a dictionary representation of the documents.
dictionary = Dictionary(corpora)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
print('Data prepared')

corpus = [dictionary.doc2bow(doc) for doc in corpora]

print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Data prepared
Number of unique tokens: 8660
Number of documents: 77782


In [260]:
corpus

[[(0, 9),
  (1, 4),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 2),
  (6, 11),
  (7, 5),
  (8, 2),
  (9, 2),
  (10, 1),
  (11, 6),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 2),
  (17, 1),
  (18, 18),
  (19, 1),
  (20, 5),
  (21, 3),
  (22, 1),
  (23, 3),
  (24, 1)],
 [(19, 1),
  (25, 1),
  (26, 3),
  (27, 3),
  (28, 1),
  (29, 6),
  (30, 2),
  (31, 1),
  (32, 2),
  (33, 2),
  (34, 1),
  (35, 3),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 7),
  (47, 2),
  (48, 1)],
 [(17, 1),
  (26, 3),
  (28, 3),
  (29, 3),
  (32, 2),
  (35, 1),
  (40, 6),
  (43, 2),
  (44, 1),
  (47, 1),
  (49, 1),
  (50, 2),
  (51, 1),
  (52, 2),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 2),
  (60, 1),
  (61, 5),
  (62, 2),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 3),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 3),
  (77, 2),
  (78, 1),
  (79, 2),
  (80

### 3.A) Text Fréquency - Inverse Document Frequency (tf-idf)

On peut transformer un texte en ses features soit :
- En utilisant une représentation de comptage creuse - fréquence d’apparition du mot dans un document, ou vecteur tf-idf d’un document, etc.
- En utilisant une représentation type word2vec dense - dans laquelle le mot possède une représentation dans un espace qui le positionne en fonction des mots adjacents

Ici on veut carractériser les textes par un tags on veut donc rechercher des mots-clef. A cette fin une représentation de comptage creuse semble plus pertinente.

In [14]:
def parallelize_dataframe(df, func, options=None, num_partitions=None):
    print('debug par_df 1: ', options, num_partitions)
    num_cores = multiprocessing.cpu_count()
    
    if num_partitions is None:
        a = np.array_split(df, num_cores)
    else :
        a = np.array_split(df, num_partitions)
    print('debug par_df 2')

    rc = ipp.Cluster(n=4).start_and_connect_sync()
    dview=rc[:]
    
    if options is None:
        print('debug par_df 3.1')
        result = dview.map_sync(func, df)
    else :
        print('debug par_df 3.2')
        result = dview.map_sync(func, df, options)
        
    print('debug par_df 4')
    return result

def f_return(text):
    return text

def TFidF(corpora, rejected=None):
    print('debug TFidF 1')
    vectorizer = skl.feature_extraction.text.TfidfVectorizer(
        token_pattern=None,
        stop_words = rejected,
        analyzer = 'word',
        preprocessor = f_return,
        tokenizer = f_return
        )
    print('debug TFidF 2')
    result = vectorizer.fit_transform(corpora)
    print('debug TFidF 3')
    features = vectorizer.get_feature_names()
    print('debug TFidF 4')
    return features, result

In [16]:
def test_parrallelisation(labels, feat):
#     bar = progress_bar(40, 'test', time_lim=1)
#     lenght = len(df)
#     for index, lab in enumerate(labels):
#         progress = index / df
#         bar.update_progress(progress)
    print(df[feat])
    return

In [ ]:
df = DataModel.copy()
t0=time.time()
parallelize_dataframe(df, test_parrallelisation, 'Tags')
print (time.time() - t0)

debug par_df 1:  Tags None
debug par_df 2
debug par_df 3.2


In [15]:
# rejected = u_doc
df = DataTagged.copy()
corpora = df['Title'] + df['Body']

 # print('Corpora :', corpus)
 # print(len(corpora),'\n')

vectorizer = skl.feature_extraction.text.TfidfVectorizer(
    token_pattern=None,
#     stop_words = rejected,
    analyzer = 'word',
    preprocessor = f_return,
    tokenizer = f_return
    )

t0=time.time()
X = vectorizer.fit_transform(corpora)
features = vectorizer.get_feature_names()
print ('Fitting Time : {} seconds'.format(time.time() - t0) )

 # print('TF-iDF : ', vectorizer.get_feature_names())
DataBow = pd.DataFrame(X.todense(), columns=features)
DataBow.head()

Fitting Time : 3.026285409927368 seconds


,a,aa,aab,aabaa,aabb,aabbc,aabbccddeeffgghhiijjkkllmmnnooppqqrrssttuuvvwwxxyyzz,aabc,aabccdd,aac,aacb,aacd,aacute,aad,aadapter,aadb,aadc,aadhaar,aadhar,aadharno,aadsts,aae,aaeb,aaf,aafcd,aafd,aafeb,aaff,aag,aal,aam,aan,aantal,aap,aapl,aapt,aar,aarch,aardvark,aaron,aasf,aasm,aat,aatg,aatrox,aav,aaw,aax,aaz,aazgjwfdmvna,ab,aba,abaa,abab,ababa,abac,abandon,abandoned,abap,abaqus,abarth,abb,abba,abbc,abbey,abbr,abbrev,abbreviated,abbreviation,abby,abc,abcabc,abcabcabc,abcaccounting,abcb,abcc,abcd,abcde,abcdeab,abcdef,abcdefabcdef,abcdefg,abcdefgh,abcdefghijk,abcdefghijklmnopqrstuvwxyz,abcdefghijklmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyz,abce,abcf,abcmeta,abcxyz,abd,abda,abdc,...,zoneinfo,zoneoffset,zonetask,zoo,zookeeper,zoom,zoomcontrol,zoomed,zoomin,zooming,zoomlevel,zoomtype,zope,zopetransactionevents,zorder,zorg,zorro,zos,zoulias,zout,zp,zpc,zpos,zposition,zprofile,zq,zqz,zr,zscore,zsh,zshell,zshrc,zsi,zst,zstack,zstd,zstream,zt,zta,zte,zti,ztm,zts,ztv,zu,zuckerberg,zugangsdaten,zulu,zum,zur,zurb,zurck,zurcksetzen,zuul,zv,zvi,zvj,zw,zwclose,zwk,zwopenfile,zx,zxb,zxc,zxcvbn,zxf,zxg,zxing,zxjzaw,zxm,zxvf,zxy,zy,zyb,zybooks,zygote,zygoteinit,zyk,zynq,zyqqbmm,zyw,zyx,zyxwvutsrqponmlkjihgfedcba,zyyw,zyz,zz,zza,zzj,zzs,zzttmi,zzx,zzz,zzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [18]:
df_freq[df_freq['Words'] == 'abcdefghijklambdaisawesome']

NameError: name 'df_freq' is not defined

In [150]:
feat = 'Body'
lookedfor = [['abcdefghijklambdaisawesome']]
    
test = pd.DataFrame(DataModel[feat].apply(check_words_list, args=(lookedfor)))
test = test[test.astype(str)[feat] != '[]']
test

,Body
48955,[abcdefghijklambdaisawesome]


In [151]:
DataRaw.loc[48955, 'Body']

'<p>I\'m trying to invoke my lambda using aws cli:</p>\n\n<pre><code>$ aws lambda invoke \\\n    --function-name soc-update-dynamodb-java \\\n    --invocation-type Event \\\n    --payload file://invoke-payload.json \\\n   response.json\n</code></pre>\n\n<p>However, I\'m getting this message:</p>\n\n<blockquote>\n  <p>An error occurred (InvalidRequestContentException) when calling the Invoke operation: Could not parse request body into json: Invalid UTF-8 middle byte 0x28\n   at [Source: (byte[])"E�(�����U�슉���ޞԨ��k.....</p>\n</blockquote>\n\n<p><code>payload.json</code> content is a s3 event-like json:</p>\n\n<pre><code>{\n"Records": [\n  {\n    "eventVersion": "2.0",\n    "eventSource": "aws:s3",\n    "awsRegion": "eu-central-1",\n    "eventTime": "1970-01-01T00:00:00.000Z",\n    "eventName": "ObjectCreated:Put",\n    "userIdentity": {\n      "principalId": "EXAMPLE"\n    },\n    "requestParameters": {\n      "sourceIPAddress": "127.0.0.1"\n    },\n    "responseElements": {\n      "x-

In [19]:
col = DataBow.columns
x = []
for c in col:
#     print(c)
    x.append(len(c))

fig = plt.figure(figsize=(10,9))
plt.hist(x, bins = 186, rwidth=0.8)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# DataBow.select_dtypes(exclude=['datetime64'])

#### Gensim library

In [297]:
tfidf = TfidfModel(corpus, normalize=True)
corpus_tfidf = tfidf[corpus]
corpus_tfidf

### 3.B) Word2vec

In [268]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec

In [273]:
print(common_texts)

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]


In [288]:
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
vector = model.wv['computer']  # get numpy vector of a word
sims = model.wv.most_similar('computer', topn=10)  # get other similar words
corpus_w2v = model.wv
corpus_w2v

### 3.C) Doc2vec

In [289]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)
corpus_d2v = model.wv
corpus_d2v

### 3.D) t-SNE

In [14]:
n_view = 2
n_feat = 2559

data_pca = DataBow.copy()

Values = data_pca.values
features = data_pca.columns

pca = decomposition.PCA(n_components=n_view).fit(Values)
X_PCA = pca.transform(Values)

plt.ioff()
fig3D = plt.figure(figsize=(7,6))
if n_view == 3:
    ax = fig3D.gca(projection='3d')
    ax.scatter(X_PCA[:,0], X_PCA[:,1], X_PCA[:,2])
    ax.set_xlabel('F0', fontsize=10)
    ax.set_ylabel('F1', fontsize=10)
    ax.set_zlabel('F2', fontsize=10)
elif n_view == 2:
    ax = fig3D.gca()
    ax.scatter(X_PCA[:,0], X_PCA[:,1])
    ax.set_xlabel('F0', fontsize=10)
    ax.set_ylabel('F1', fontsize=10)
else :
    print('No Draw')

plt.show(block=False)

data_pca.describe()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,F0,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28,F29,F30,F31,F32,F33,F34,F35,F36,F37,F38,F39,F40,F41,F42,F43,F44,F45,F46,F47,F48,F49,F50,F51,F52,F53,F54,F55,F56,F57,F58,F59,F60,F61,F62,F63,F64,F65,F66,F67,F68,F69,F70,F71,F72,F73,F74,F75,F76,F77,F78,F79,F80,F81,F82,F83,F84,F85,F86,F87,F88,F89,F90,F91,F92,...,F2466,F2467,F2468,F2469,F2470,F2471,F2472,F2473,F2474,F2475,F2476,F2477,F2478,F2479,F2480,F2481,F2482,F2483,F2484,F2485,F2486,F2487,F2488,F2489,F2490,F2491,F2492,F2493,F2494,F2495,F2496,F2497,F2498,F2499,F2500,F2501,F2502,F2503,F2504,F2505,F2506,F2507,F2508,F2509,F2510,F2511,F2512,F2513,F2514,F2515,F2516,F2517,F2518,F2519,F2520,F2521,F2522,F2523,F2524,F2525,F2526,F2527,F2528,F2529,F2530,F2531,F2532,F2533,F2534,F2535,F2536,F2537,F2538,F2539,F2540,F2541,F2542,F2543,F2544,F2545,F2546,F2547,F2548,F2549,F2550,F2551,F2552,F2553,F2554,F2555,F2556,F2557,F2558
count,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,...,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04,1.000000e+04
mean,6.536265e-18,-3.316791e-18,-5.289887e-18,3.166911e-18,7.755667e-18,3.792106e-18,2.242130e-18,-3.745919e-20,-1.176576e-18,2.920580e-18,1.332528e-18,5.604718e-18,2.113761e-18,1.782970e-18,-2.956749e-18,-1.437110e-18,-4.358666e-18,1.360630e-18,-1.380666e-18,-5.967449e-20,-3.423236e-18,1.925717e-18,-9.242249e-18,3.051552e-18,-2.287753e-18,3.472396e-18,-3.072304e-18,2.138610e-18,-3.193366e-18,5.373154e-18,6.371206e-19,-8.047165e-19,-3.294023e-18,-2.917420e-18,-3.538619e-18,-1.708009e-18,-2.702786e-18,-8.342285e-19,-6.388987e-19,1.546354e-18,-3.228949e-18,-5.278937e-18,3.798611e-19,-5.398459e-19,-2.899634e-18,-1.507561e-18,-6.204239e-19,2.3

2566 features selected

## 4) Modèles non-supervisé

On veut ici déterminer les différents sujets du corpus. Pour chaque document on veut déterminer les sujets abordés.

### 4.A) Latent Dirichlet allocation (LDA)
C’est une méthode non-supervisée générative qui se base sur les hypothèses suivantes :

- Chaque document du corpus est un ensemble de mots sans ordre (bag-of-words) ;
- Chaque document m aborde un certain nombre de thèmes dans différentes proportions qui lui sont propres p(θm)
- Chaque mot possède une distribution associée à chaque thème p(ϕk). On peut ainsi représenter chaque thème par une probabilité sur chaque mot.
- zn représente le thème du mot wn

Puisque l'on a accès uniquement aux documents, on doit déterminer quels sont les thèmes, les distributions de chaque mot sur les thèmes, la fréquence d’apparition de chaque thème sur le corpus. <br>

A titre d’exemple, voici une liste des hypothèses supplémentaires qui mènent à une modélisation plus riche :

- La distribution des mots sur les thèmes évoluent avec le temps. Ce qui signifie qu’il faut créer une séquence de distribution pour chaque thème qui permet de modéliser l’évolution du thème dans le temps.
- Certains thèmes sont plus proches que d’autres. L’hypothèse d’utiliser la distribution de Dirichlet considère que les différents thèmes sont complètement indépendants alors qu’en réalité certains thèmes ont en général plus de chances d’apparaître ensemble.

#### Gensim library

In [8]:
model_results = {
    'Validation_Set': [],
    'Topics': [],
    'Alpha': [],
    'Eta': [],
    'Coherence': []
}

# Set training parameters.
# num_topics = 369
num_topics = 5
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

t0=time.time()

for num_topics in range(10,370,10):
    print('Nombre de sujets : {}'.format(num_topics))
    t1 = time.time()
    try:
        model = LdaMulticore(
            random_state=100,
            corpus = corpus,
            id2word = id2word,
            chunksize = chunksize,
        #     alpha = 'auto',
            eta = 'auto',
            iterations = iterations,
            num_topics = num_topics,
            passes = passes,
            eval_every = eval_every
        )
        top_topics = model.top_topics(corpus)
        avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
        print('Average topic coherence: %.4f.' % avg_topic_coherence)

        coherence_model_lda = CoherenceModel(
            model = model, 
            texts = corpora, 
            dictionary = dictionary,
            coherence = 'c_v'
        )
        coherence_lda = coherence_model_lda.get_coherence()
        print('Coherence Score: ', coherence_lda)

        model_results['Validation_Set'].append('No Title')
        model_results['Topics'].append(num_topics)
        model_results['Alpha'].append('default')
        model_results['Eta'].append('auto')
        model_results['Coherence'].append(coherence_lda)

        print('Processing Time : {} seconds'.format(time.time() - t1) )
        print('\n')
        
    except :
        print('Problem pour {} sujets '.format(num_topics))
        print('Processing Time : {} seconds'.format(time.time() - t1) )
        print('\n')
    
print('Over all Time : {} seconds'.format(time.time() - t0) )

Nombre de sujets : 10
Average topic coherence: -2.0704.
Coherence Score:  0.5185010429178644
Processing Time : 322.6317141056061 seconds


Nombre de sujets : 20
Average topic coherence: -2.3772.
Coherence Score:  0.521638343134403
Processing Time : 339.5404922962189 seconds


Nombre de sujets : 30
Average topic coherence: -2.5804.
Coherence Score:  0.5198390146136067
Processing Time : 355.1268241405487 seconds


Nombre de sujets : 40
Average topic coherence: -2.9681.
Coherence Score:  0.5211475637925648
Processing Time : 369.91907596588135 seconds


Nombre de sujets : 50
Average topic coherence: -3.1977.
Coherence Score:  0.5241965121637785
Processing Time : 367.232524394989 seconds


Nombre de sujets : 60
Average topic coherence: -3.5524.
Coherence Score:  0.5131414243219301
Processing Time : 378.5189116001129 seconds


Nombre de sujets : 70
Average topic coherence: -3.7023.
Coherence Score:  0.5176110981693821
Processing Time : 428.8258202075958 seconds


Nombre de sujets : 80
Averag

In [10]:
plt.plot(model_results['Topics'], model_results['Coherence'], '-o', label = 'Eta')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Ici le nombre optimal de sujets est 50.

In [251]:
model_results = {
    'Validation_Set': [],
    'Topics': [],
    'Alpha': [],
    'Eta': [],
    'Coherence': []
}

# Set training parameters.
# num_topics = 369
num_topics = 50
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

t0=time.time()

print('Nombre de sujets : {}'.format(num_topics))

for eta in range(1,10,2):
    eta = 0.1*eta +0.01
    for alpha in range(1,10,2):
        alpha = 0.1*alpha +0.01
        print('Hyperparrametres : ({0:.2f}, {1:.2f})'.format(alpha, eta))
        t1 = time.time()
        try:
            model = LdaMulticore(
                random_state=100,
                corpus = corpus,
                id2word = id2word,
                chunksize = chunksize,
                alpha = alpha,
                eta = eta,
                iterations = iterations,
                num_topics = num_topics,
                passes = passes,
                eval_every = eval_every
            )
            top_topics = model.top_topics(corpus)
            avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
            print('Average topic coherence: %.4f.' % avg_topic_coherence)

            coherence_model_lda = CoherenceModel(
                model = model, 
                texts = corpora, 
                dictionary = dictionary,
                coherence = 'c_v'
            )
            coherence_lda = coherence_model_lda.get_coherence()
            print('Coherence Score: ', coherence_lda)

            model_results['Validation_Set'].append('No Title')
            model_results['Topics'].append(num_topics)
            model_results['Alpha'].append(alpha)
            model_results['Eta'].append(eta)
            model_results['Coherence'].append(coherence_lda)

            print('Processing Time : {} seconds'.format(time.time() - t1) )
            print('\n')

        except :
            print('Problem pour {} sujets '.format(num_topics))
            print('Processing Time : {} seconds'.format(time.time() - t1) )
            print('\n')
    
print('Over all Time : {} seconds'.format(time.time() - t0) )

Nombre de sujets : 50
Hyperparrametres : (0.11, 0.11)
Average topic coherence: -3.1256.
Coherence Score:  0.5414509701092045
Processing Time : 400.6065299510956 seconds


Hyperparrametres : (0.31, 0.11)
Average topic coherence: -3.4308.
Coherence Score:  0.5122584796162262
Processing Time : 429.3266005516052 seconds


Hyperparrametres : (0.51, 0.11)
Average topic coherence: -3.2424.
Coherence Score:  0.5388704326196987
Processing Time : 347.1789581775665 seconds


Hyperparrametres : (0.71, 0.11)
Average topic coherence: -3.2694.
Coherence Score:  0.5352727632371423
Processing Time : 305.8741512298584 seconds


Hyperparrametres : (0.91, 0.11)
Average topic coherence: -3.1687.
Coherence Score:  0.5306691589210092
Processing Time : 289.0265517234802 seconds


Hyperparrametres : (0.11, 0.31)
Average topic coherence: -3.2054.
Coherence Score:  0.5229831980939812
Processing Time : 370.0806214809418 seconds


Hyperparrametres : (0.31, 0.31)
Average topic coherence: -3.4287.
Coherence Score:  

Ici on converge vers un modèle.

In [252]:
# for doc in corpus:
bar = progress_bar(10, 'treatement', 1e-2)
num_topics = 50
columns = ["Topic{}".format(i) for i in range(0, num_topics, 1)]
docnames = ["Doc{}".format(0)]

doc_topic = pd.DataFrame(columns=columns)

bar.update_progress(0)
for i, doc in enumerate(corpus):
    progress = i/len(corpus)
    serie = pd.Series()
    for item in model.get_document_topics(doc):
    #     print(item)
        feat = 'Topic{0:.0f}'.format(item[0])
    #     print(type(item[1]))
        serie[feat] = item[1]
    doc_topic = doc_topic.append(serie, ignore_index = True)
    bar.update_progress(progress)

doc_topic

<ipython-input-252-2a6910c78ba5>:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie = pd.Series()


Progress treatement: [########################################] 99.00% 1:20:41.309158

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,Topic20,Topic21,Topic22,Topic23,Topic24,Topic25,Topic26,Topic27,Topic28,Topic29,Topic30,Topic31,Topic32,Topic33,Topic34,Topic35,Topic36,Topic37,Topic38,Topic39,Topic40,Topic41,Topic42,Topic43,Topic44,Topic45,Topic46,Topic47,Topic48,Topic49,Topic50,Topic51,Topic52,Topic53,Topic54,Topic55,Topic56,Topic57,Topic58,Topic59,Topic60,Topic61,Topic62,Topic63,Topic64,Topic65,Topic66,Topic67,Topic68,Topic69,Topic70,Topic71,Topic72,Topic73,Topic74,Topic75,Topic76,Topic77,Topic78,Topic79,Topic80,Topic81,Topic82,Topic83,Topic84,Topic85,Topic86,Topic87,Topic88,Topic89,Topic90,Topic91,Topic92,...,Topic267,Topic268,Topic269,Topic270,Topic271,Topic272,Topic273,Topic274,Topic275,Topic276,Topic277,Topic278,Topic279,Topic280,Topic281,Topic282,Topic283,Topic284,Topic285,Topic286,Topic287,Topic288,Topic289,Topic290,Topic291,Topic292,Topic293,Topic294,Topic295,Topic296,Topic297,Topic298,Topic299,Topic300,Topic301,Topic302,Topic303,Topic304,Topic305,Topic306,Topic307,Topic308,Topic309,Topic310,Topic311,Topic312,Topic313,Topic314,Topic315,Topic316,Topic317,Topic318,Topic319,Topic320,Topic321,Topic322,Topic323,Topic324,Topic325,Topic326,Topic327,Topic328,Topic329,Topic330,Topic331,Topic332,Topic333,Topic334,Topic335,Topic336,Topic337,Topic338,Topic339,Topic340,Topic341,Topic342,Topic343,Topic344,Topic345,Topic346,Topic347,Topic348,Topic349,Topic350,Topic351,Topic352,Topic353,Topic354,Topic355,Topic356,Topic357,Topic358,Topic359
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027869,NaN,0.014432,NaN,0.049295,0.016701,0.022718,NaN,NaN,NaN,NaN,NaN,0.041337,NaN,0.019644,NaN,0.015863,NaN,NaN,NaN,0.172819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.078635,0.013436,0.016144,NaN,0.213937,NaN,NaN,0.017621,0.013050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.010954,0.015573,0.010245,0.010284,0.010182,0.020608,0.011283,0.011744,0.019706,0.015804,0.010487,0.010148,0.010577,NaN,0.011646,0.038098,0.010533,NaN,NaN,0.010179,0.015761,NaN,NaN,0.010187,0.371881,0.010930,0.010094,0.010048,NaN,0.010048,0.022313,0.011734,NaN,0.014895,0.010687,NaN,0.011318,NaN,NaN,0.010780,NaN,0.013937,0.010173,NaN,0.010500,0.010458,0.010228,0.010392,0.046502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.017001,0.203264,NaN,NaN,NaN,NaN,NaN,0.010493,NaN,NaN,NaN,NaN,NaN,NaN,0.039807,NaN,NaN,0.081560,NaN,NaN,NaN,NaN,0.011767,0.011837,0.233645,NaN,NaN,0.014212,NaN,NaN,0.026992,0.013033,NaN,0.032470,0.014059,NaN,NaN,0.011892,NaN,NaN,NaN,0.014076,NaN,NaN,NaN,NaN,0.039830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [205]:
def Doc2tags(row, words, bar=None, Lenght=None):
    if Lenght is not None:
        progress = row.name/Lenght
    serie = pd.Series()
    for tag in row['Tags']:
        if tag in words:
            serie[tag] = 1
    if bar is not None:
        bar.update_progress(progress)
    return serie

In [253]:
df = DataTagged.copy()
doc_tag = pd.DataFrame(columns=words)
bar = progress_bar(10, 'treatement', 1e-2)

doc_tag = pd.DataFrame(columns=words)

bar.set_progress(0)

doc_tag = doc_tag.append(
    df.apply(
        Doc2tags,
        args=(words,bar,len(df)),
        axis = 1
    )
)

doc_tag

Progress treatement: [#---------------------------------------] 2.00% 0:00:02.134317

<ipython-input-205-93c4f159465b>:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  serie = pd.Series()


Progress treatement: [#########################################] 102.00% 0:01:47.097740

In [255]:
doc_tag

,python,javascript,c++,c#,java,c,php,html,reactjs,sql,arrays,python-3.x,node.js,android,css,mysql,angular,laravel,json,r,sql-server,jquery,swift,pandas,typescript,excel,vba,ios,regex,algorithm,string,bash,flutter,.net,django,linux,asp.net-core,docker,list,spring-boot,vue.js,amazon-web-services,firebase,numpy,pointers,multithreading,postgresql,.net-core,function,spring,dataframe,windows,react-native,mongodb,azure,loops,ajax,performance,google-apps-script,kotlin,oracle,powershell,asp.net,visual-studio,git,dart,go,csv,for-loop,dictionary,linq,class,c++11,shell,kubernetes,database,express,async-await,sorting,tkinter,assembly,file,api,google-sheets,selenium,vb.net,wpf,recursion,unity3d,winforms,winapi,c++17,google-cloud-platform,...,scikit-learn,error-handling,codeigniter,firebase-authentication,django-models,dependency-injection,makefile,time,flutter-layout,collections,heroku,react-router,angular8,indexing,next.js,iis,web,utf-8,security,material-ui,.htaccess,split,google-bigquery,design-patterns,discord.py,testing,ionic-framework,unicode,button,ssh,embedded,email,deep-learning,reference,scanf,asp.net-web-api,multiprocessing,select,graph,jupyter-notebook,junit,pygame,integer,flexbox,import,scipy,fortran,uwp,serialization,time-complexity,c-strings,dockerfile,hashmap,mysqli,postman,dynamic,terraform,ffmpeg,type-conversion,server,replace,mvvm,asp.net-core-webapi,xamarin.android,apache-spark-sql,merge,hash,twitter-bootstrap,jenkins,stack,xpath,android-recyclerview,angular-material,clang,asp.net-core-3.1,duplicates,opengl,jdbc,anaconda,pyqt5,use-effect,g++,session,terminal,google-drive-api,google-maps,search,ansible,count,data.table,laravel-7,iteration,electron
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [254]:
# np.asmatrix(doc_topic.values)
doc_topic = doc_topic.fillna(0)
# np.asmatrix(doc_tag.values)
doc_tag = doc_tag.fillna(0)

Tag_topic = pd.DataFrame(np.dot(doc_topic.T,doc_tag), columns = doc_tag.columns, index = doc_topic.columns)
Tag_topic

,python,javascript,c++,c#,java,c,php,html,reactjs,sql,arrays,python-3.x,node.js,android,css,mysql,angular,laravel,json,r,sql-server,jquery,swift,pandas,typescript,excel,vba,ios,regex,algorithm,string,bash,flutter,.net,django,linux,asp.net-core,docker,list,spring-boot,vue.js,amazon-web-services,firebase,numpy,pointers,multithreading,postgresql,.net-core,function,spring,dataframe,windows,react-native,mongodb,azure,loops,ajax,performance,google-apps-script,kotlin,oracle,powershell,asp.net,visual-studio,git,dart,go,csv,for-loop,dictionary,linq,class,c++11,shell,kubernetes,database,express,async-await,sorting,tkinter,assembly,file,api,google-sheets,selenium,vb.net,wpf,recursion,unity3d,winforms,winapi,c++17,google-cloud-platform,...,scikit-learn,error-handling,codeigniter,firebase-authentication,django-models,dependency-injection,makefile,time,flutter-layout,collections,heroku,react-router,angular8,indexing,next.js,iis,web,utf-8,security,material-ui,.htaccess,split,google-bigquery,design-patterns,discord.py,testing,ionic-framework,unicode,button,ssh,embedded,email,deep-learning,reference,scanf,asp.net-web-api,multiprocessing,select,graph,jupyter-notebook,junit,pygame,integer,flexbox,import,scipy,fortran,uwp,serialization,time-complexity,c-strings,dockerfile,hashmap,mysqli,postman,dynamic,terraform,ffmpeg,type-conversion,server,replace,mvvm,asp.net-core-webapi,xamarin.android,apache-spark-sql,merge,hash,twitter-bootstrap,jenkins,stack,xpath,android-recyclerview,angular-material,clang,asp.net-core-3.1,duplicates,opengl,jdbc,anaconda,pyqt5,use-effect,g++,session,terminal,google-drive-api,google-maps,search,ansible,count,data.table,laravel-7,iteration,electron
Topic0,95.272437,108.758096,68.872931,64.851239,41.059548,37.704489,24.927681,25.897529,28.950690,16.968955,35.436735,20.311741,17.328775,14.164940,16.044481,9.942679,17.748350,13.844523,21.363956,9.994491,10.207549,9.926984,271.911961,9.071804,15.853654,6.915982,5.739724,189.690473,11.408243,11.278407,10.074948,7.510018,8.609015,8.925745,9.556549,6.575208,6.913335,2.940119,8.956168,3.808363,6.527238,4.468040,19.874504,5.783684,6.029814,6.734128,5.156363,5.464635,7.946841,3.314862,3.649229,5.263233,13.008690,4.297262,3.870750,5.811465,2.985641,5.548993,3.677575,4.812312,3.973394,3.369586,3.964944,4.340235,4.275625,4.333325,14.353987,3.130046,5.154763,6.495096,4.316083,4.631435,4.195226,3.338854,2.141995,3.908728,2.392503,5.660007,5.191876,2.233932,5.018140,3.402367,4.232902,3.009083,2.407555,2.127499,3.166073,5.089844,3.655110,3.113749,3.463662,4.318798,2.202260,...,0.471518,1.613592,0.798460,3.807638,0.954316,1.021809,0.859454,1.000416,0.882111,1.220779,0.514168,0.633447,1.172074,0.782823,0.668139,0.658659,0.974719,0.901324,0.932361,0.651103,0.659534,0.837617,0.748027,1.817723,0.489870,1.140961,0.911497,1.004534,1.600601,0.527658,1.166145,0.462816,0.314763,1.121899,0.732588,0.696532,1.024335,0.660387,0.627777,0.781670,0.400348,0.516337,0.848143,0.753347,1.014088,1.515037,0.664633,1.058819,0.767605,1.027667,0.764320,0.416131,0.531523,0.292624,0.511104,1.279142,0.381692,0.965079,0.819335,0.565116,0.573302,2.543603,0.451872,0.894239,0.561731,0.885525,0.807229,0.407804,0.581392,0.670503,0.690502,0.554888,1.216571,1.050577,0.732034,0.863194,0.599427,0.469720,0.486748,0.880543,0.695000,0.767881,0.393177,0.710368,0.513561,0.724849,0.676966,0.449881,0.591120,0.414644,0.642112,0.817857,0.779682
Topic1,164.881595,390.808274,60.554267,115.882416,59.052700,34.736140,221.414807,243.486064,75.135827,30.605875,35.448081,33.152814,36.865232,18.671228,71.158458,45.435635,62.237771,69.241590,34.045988,14.235934,12.535176,152.303190,14.194954,10.958807,31.184848,14.233533,14.156927,11.134812,19.102240,10.895181,12.839928,11.744136,9.904590,14.036514,46.998676,7.997328,20.932115,4.956160,9.035444,9.248664,29.423330,7.215910,8.505109,4.820103,4.466560,5.866695,5.956521,8.162102,10.639548,7.761571,4.838115,6.173964,7.588305,9.464462,5.422248,6.940035,73.893759,4.652829,12.307819,4.380832,4.053376,5.2

In [298]:
values = []
words = []

for topic, obj in enumerate(top_topics):
    print('topics : ', topic)
    for truc in obj:
        if type(truc) is list:
            for item in truc:
                values.append(item[0])
                words.append(item[1])
            print('Words : ', words)
        else :
            print('Coherence : ', truc)
            
        values = []
        words = []

topics :  0
Words :  ['div', 'class', 'button', 'style', 'px', 'cs', 'html', 'display', 'color', 'text', 'span', 'card', 'href', 'background', 'width', 'body', 'border', 'click', 'li', 'container']
Coherence :  -2.0822865600815423
topics :  1
Words :  ['form', 'page', 'input', 'post', 'html', 'td', 'search', 'id', 'th', 'tr', 'class', 'control', 'submit', 'button', 'ajax', 'text', 'php', 'jquery', 'title', 'placeholder']
Coherence :  -2.2329279101234594
topics :  2
Words :  ['http', 'request', 'response', 'api', 'com', 'header', 'content', 'url', 'status', 'get', 'application', 'post', 'access', 'body', 'curl', 'error', 'browser', 'fetch', 'cache', 'localhost']
Coherence :  -2.2569806230510085
topics :  3
Words :  ['state', 'event', 'component', 'import', 'const', 'react', 'default', 'export', 'action', 'return', 'prop', 'store', 'classname', 'render', 'constructor', 'payload', 'hook', 'router', 'app', 'setstate']
Coherence :  -2.393313481900229
topics :  4
Words :  ['file', 'line', 'p

#### Scikit-learn library

In [21]:
class MyLDAWithPerplexityScorer(skl.decomposition.LatentDirichletAllocation):

    def score(self, X, y=None):

        # You can change the options passed to perplexity here
        score = super(MyLDAWithPerplexityScorer, self).perplexity(X, sub_sampling=False)

        # Since perplexity is lower for better, so we do negative
        return -1*score

In [111]:
max_iter = 10
no_top_words = 10

search_params = {
    'n_components': range(1,10),
    'learning_decay': [.5, .7, .9]
}

# data_classique = data_classique.select_dtypes(exclude=['datetime64'])
Values = DataBow.values
features_names = DataBow.columns

lda = MyLDAWithPerplexityScorer(
#     n_components = n_topics,
    max_iter = max_iter, 
    learning_method = 'online',
    learning_offset = 50,
#     learning_decay = 0.7,
    random_state = 0,
    verbose = 0
)

model = GridSearchCV(
    n_jobs = -1,
    cv = None,
    error_score = 'raise',
    estimator = lda,
    param_grid = search_params,
    scoring = None,
    verbose = 2
)

model.fit(Values)

print('\n')
for topic_idx, topic in enumerate(model.best_estimator_.components_):
    print('Topic %d:' % (topic_idx))
    print(' '.join([features_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

print('\n')
# Log Likelyhood: Higher the better
print("Log Likelihood: ", model.best_estimator_.score(Values))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", model.best_estimator_.perplexity(Values))


# See model parameters
print(model.best_estimator_.get_params())

Fitting 5 folds for each of 27 candidates, totalling 135 fits


Topic 0:
error file data class python name value code app import


Log Likelihood:  -5108.607174291086
Perplexity:  5108.607174291086
{'batch_size': 128, 'doc_topic_prior': None, 'evaluate_every': -1, 'learning_decay': 0.9, 'learning_method': 'online', 'learning_offset': 50, 'max_doc_update_iter': 100, 'max_iter': 10, 'mean_change_tol': 0.001, 'n_components': 1, 'n_jobs': None, 'perp_tol': 0.1, 'random_state': 0, 'topic_word_prior': None, 'total_samples': 1000000.0, 'verbose': 0}


In [157]:
model.cv_results_
params = model.cv_results_['params']
scores_mean = model.cv_results_['mean_test_score']
grid_param_1 = []
name_param_1 = 'N_components'
grid_param_2 = []
name_param_2 = 'Learning_decay'

for i, row in enumerate(params):
    grid_param_1.append(row['n_components'])
    grid_param_2.append(row['learning_decay'])

dfpar = pd.DataFrame({
    'Param1' : grid_param_1,
    'Param2' : grid_param_2,
    'Mean_score' : scores_mean
})
u_par2 = dfpar['Param2'].unique()
# # Plot Grid search scores
_, ax = plt.subplots(1,1)

# # Param1 is the X-axis, Param 2 is represented as a different curve (color line)
for idx, val in enumerate(u_par2):
    df2 = dfpar[dfpar['Param2'] == val]
    ax.plot(dfpar['Param1'], dfpar['Mean_score'], '-o', label= name_param_2 + ': ' + str(val))

ax.set_title("Grid Search Scores", fontsize=20, fontweight='bold')
ax.set_xlabel(name_param_1, fontsize=16)
ax.set_ylabel('CV Average Score', fontsize=16)
ax.legend(loc="best", fontsize=15)
ax.grid('on')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 4.B) Negative Matrix Factorisation (NMF)

## 5) Modèle supervisé

### 5.A) One versus Rest

In [ ]:
# logistic regression for multi-class classification using a one-vs-rest

# define dataset
X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=5,
    n_redundant=5,
    n_classes=3,
    random_state=1
)

# define model
model = LogisticRegression()
# define the ovr strategy
ovr = OneVsRestClassifier(model)
# fit model
ovr.fit(X, y)
# make predictions
yhat = ovr.predict(X)

### 5.B) One versus One

### 5.C) Chain